In [3]:
import pandas as pd
import numpy as np
from pandas import Series , DataFrame
from xgboost.sklearn import XGBClassifier

In [2]:
acc=pd.read_csv(r'C:\Users\Vaibhav Mathur\Desktop\Affine\AffineData\Dataset\training dataset\Accounts.csv')
test=pd.read_csv(r'C:\Users\Vaibhav Mathur\Desktop\Affine\AffineData\Dataset\Test dataset\Test_Data.csv')
acc_prop=pd.read_csv(r'C:\Users\Vaibhav Mathur\Desktop\Affine\AffineData\Dataset\training dataset\Accounts_properties.csv')
prop=pd.read_csv(r'C:\Users\Vaibhav Mathur\Desktop\Affine\AffineData\Dataset\Property dataset\Properties.csv')
oppo=pd.read_csv(r'C:\Users\Vaibhav Mathur\Desktop\Affine\AffineData\Dataset\training dataset\Opportunities.csv')

In [3]:
dealtoprop = pd.read_csv(r'C:\Users\Vincy Sagar\Desktop\Affine\AffineData\Dataset\training dataset\Deals_to_properties.csv')

In [4]:
acc.head()

,id_accs,active_deals,activity_count,buyer_book,servicing_contract,investor_type,cmbs,consultant,correspondent,foreign,master_servicer,lender_book,loan_sales_book,loan_servicing,num_deals_as_client,num_deals_as_investor,number_of_properties,number_of_related_deals,number_of_related_properties,number_of_won_deals_as_client
0,0012A000023XlCCQA0,1,2367,True,False,Private Investor/Developer,False,False,False,False,False,False,False,False,0,1,1,2,1,0
1,0012A000023XmOxQAK,9,7974,True,False,REIT,False,False,False,False,False,True,True,False,7,2,77,18,85,7
2,0012A000023YLqwQAG,891,50161,True,True,Bank / Thrift,False,False,False,False,False,True,True,False,40,851,693,996,1167,37
3,0012A000023XlNjQAK,34,14631,True,False,Conduit/CMBS,True,False,False,True,False,True,True,False,0,34,0,45,14,0
4,0012A000023YBFRQA4,283,194195,True,False,Life Insurance Company,False,False,False,False,False,True,True,False,63,220,393,383,591,46


In [5]:
acc.shape

(2756, 20)

In [6]:
acc.describe()

,active_deals,activity_count,num_deals_as_client,num_deals_as_investor,number_of_properties,number_of_related_deals,number_of_related_properties,number_of_won_deals_as_client
count,2756.000000,2756.000000,2756.000000,2756.000000,2756.000000,2756.000000,2756.000000,2756.000000
mean,16.461538,9460.461176,6.029390,10.450290,21.382075,21.567126,28.296807,4.881713
std,73.668032,20528.690928,17.879478,68.893548,69.984153,81.063531,84.415697,15.555176
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
25%,1.000000,299.000000,0.000000,1.000000,1.000000,1.000000,2.000000,0.000000
50%,3.000000,2354.000000,0.000000,1.000000,5.000000,4.000000,7.000000,0.000000
75%,10.000000,9818.250000,5.000000,4.000000,16.000000,16.000000,21.000000,3.000000
max,2215.000000,217301.000000,333.000000,2205.000000,1335.000000,2137.000000,1479.000000,330.000000


In [7]:
######################################Here we convert the types to integer of certain columns wich are of the type bool
acc['buyer_book'] = acc['buyer_book'].astype(int)
acc['servicing_contract'] = acc['servicing_contract'].astype(int)
acc['cmbs'] = acc['cmbs'].astype(int)
acc['consultant'] = acc['consultant'].astype(int)
acc['correspondent'] = acc['correspondent'].astype(int)
acc['foreign'] = acc['foreign'].astype(int)
acc['master_servicer'] = acc['master_servicer'].astype(int)
acc['lender_book'] = acc['lender_book'].astype(int)
acc['loan_sales_book'] = acc['loan_sales_book'].astype(int)
acc['loan_servicing'] = acc['loan_servicing'].astype(int)

In [8]:
##############################################Here we create dummy for the only object dtype column
dummies_acc = pd.get_dummies(acc['investor_type'],prefix='investor_type')
acc = pd.concat([acc,dummies_acc],axis=1)
acc.drop('investor_type',axis=1,inplace=True)

In [9]:
prop.head(3)

,id_props,building_status,building_tax_expenses,id_deals,city,class,county,sale_status,portfolio,market,...,price_per_sq_ft,price_per_unit,property_type_1,region__c,sale_amount__c,sale_date__c,size_acres,size_sf,size_units,year_built
0,a0I2A00000XQGKlUAP,Existing,2009 Tax @ $1.25/sf,a0I2A00000XQGKlUAP,City-0488,Class B,County-0172,N,False,Market-0059,...,46.59,18350000.00,Industrial,Southwest,18350000.00,07-10-2015,3.760,393862.0,1.0,1980.0
1,a0I2A00000XQGKnUAP,NaN,NaN,a0I2A00000XQGKnUAP,City-1975,Class A,County-0465,Y,True,Market-0239,...,1141.55,2109090.91,Multi-Housing,Southeast,21440582.78,12/31/2016,19.600,508080.0,275.0,2014.0
2,a0I2A00000XQGKoUAP,Existing,"2016 Tax @ $5.63/sf; 2007 Ops @ $19.58/sf, 201...",a0I2A00000XQGKoUAP,City-1415,Class B,County-0452,N,False,Market-0165,...,1506.65,NaN,Retail,Northeast,27541555.35,9/30/2016,0.266,18280.0,NaN,1902.0


In [10]:
prop.shape

(18298, 24)

In [205]:
prop['region__c'].unique()

array(['Southwest', 'Southeast', 'Northeast', 'West', 'Midwest',
       'Mid-Atlantic'], dtype=object)

In [203]:
prop['region__c'].fillna('Southwest', inplace=True)

In [206]:
prop['portfolio'].unique()

array([False,  True])

In [207]:
prop['sale_status'].unique()

array(['N', 'Y', nan], dtype=object)

In [208]:
prop['sale_status'].describe()

count     18105
unique        2
top           N
freq      16501
Name: sale_status, dtype: object

In [209]:
prop['region__c'].fillna('N', inplace=True)

In [145]:
acc.head()

,id_accs,active_deals,activity_count,buyer_book,servicing_contract,cmbs,consultant,correspondent,foreign,master_servicer,...,investor_type_Life Insurance Company,investor_type_Pension Fund,investor_type_Private Investor/Developer,investor_type_Property Lender,investor_type_Property Owner,investor_type_REIT,investor_type_Sovereign Wealth Fund,investor_type_Special Servicer,investor_type_Tenants in Common,investor_type_Unknown
0,0012A000023XlCCQA0,1,2367,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,0012A000023XmOxQAK,9,7974,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,0012A000023YLqwQAG,891,50161,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0012A000023XlNjQAK,34,14631,1,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,0012A000023YBFRQA4,283,194195,1,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [210]:
##########We merge the acc_prop and acc dataframe

ap_acc=acc_prop.merge(acc, on=['id_accs'], how='left')

In [12]:
ap_acc.shape

(70350, 40)

In [13]:
ap_acc.head()

,id,id_deals,id_accs,id_props,active_deals,activity_count,buyer_book,servicing_contract,cmbs,consultant,...,investor_type_Life Insurance Company,investor_type_Pension Fund,investor_type_Private Investor/Developer,investor_type_Property Lender,investor_type_Property Owner,investor_type_REIT,investor_type_Sovereign Wealth Fund,investor_type_Special Servicer,investor_type_Tenants in Common,investor_type_Unknown
0,a062A00001Whq5tQAB,a062A00001Whq5tQAB,0012A000023XlCCQA0,a0I2A00000XQKwXUAX,1,2367,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,a062A00001VmP7EQAV,a062A00001VmP7EQAV,0012A000023XmOxQAK,a0I2A00000XvUEwUAN,9,7974,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,a062A00001VmQByQAN,a062A00001VmQByQAN,0012A000023XmOxQAK,a0I2A00000XwQ13UAF,9,7974,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,a062A00001VmQC2QAN,a062A00001VmQC2QAN,0012A000023XmOxQAK,a0I2A00000XwZyBUAV,9,7974,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,a062A00001VmW8wQAF,a062A00001VmW8wQAF,0012A000023XmOxQAK,a0I2A00000ZGZClUAP,9,7974,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [14]:
#########Now merge the dataframe prop column id_prop

ap_acc_prop=ap_acc.merge(prop[['id_props','region__c']], on=['id_props'],how='left' )

In [15]:
ap_acc_prop.shape

(70350, 41)

In [48]:
ap_acc_prop.columns

Index(['id', 'id_deals', 'id_accs', 'id_props', 'active_deals',
       'activity_count', 'buyer_book', 'servicing_contract', 'cmbs',
       'consultant', 'correspondent', 'foreign', 'master_servicer',
       'lender_book', 'loan_sales_book', 'loan_servicing',
       'num_deals_as_client', 'num_deals_as_investor', 'number_of_properties',
       'number_of_related_deals', 'number_of_related_properties',
       'number_of_won_deals_as_client', 'investor_type_Account',
       'investor_type_Advisor/Equity Fund', 'investor_type_Agency',
       'investor_type_Bank / Thrift', 'investor_type_Conduit/CMBS',
       'investor_type_Corporate Investor/User',
       'investor_type_Credit/Specialty Finance Company',
       'investor_type_Endowment', 'investor_type_Life Insurance Company',
       'investor_type_Pension Fund',
       'investor_type_Private Investor/Developer',
       'investor_type_Property Lender', 'investor_type_Property Owner',
       'investor_type_REIT', 'investor_type_Sovereign W

In [16]:
te_region=pd.DataFrame(ap_acc_prop[ap_acc_prop['region__c'].isnull()])
tr_region=pd.DataFrame(ap_acc_prop[ap_acc_prop['region__c'].notnull()])

In [17]:
te_region.head()

,id,id_deals,id_accs,id_props,active_deals,activity_count,buyer_book,servicing_contract,cmbs,consultant,...,investor_type_Pension Fund,investor_type_Private Investor/Developer,investor_type_Property Lender,investor_type_Property Owner,investor_type_REIT,investor_type_Sovereign Wealth Fund,investor_type_Special Servicer,investor_type_Tenants in Common,investor_type_Unknown,region__c
2,a062A00001VmQByQAN,a062A00001VmQByQAN,0012A000023XmOxQAK,a0I2A00000XwQ13UAF,9,7974,1,0,0,0,...,0,0,0,0,1,0,0,0,0,NaN
3,a062A00001VmQC2QAN,a062A00001VmQC2QAN,0012A000023XmOxQAK,a0I2A00000XwZyBUAV,9,7974,1,0,0,0,...,0,0,0,0,1,0,0,0,0,NaN
4,a062A00001VmW8wQAF,a062A00001VmW8wQAF,0012A000023XmOxQAK,a0I2A00000ZGZClUAP,9,7974,1,0,0,0,...,0,0,0,0,1,0,0,0,0,NaN
5,a062A00001VmW8xQAF,a062A00001VmW8xQAF,0012A000023XmOxQAK,a0I2A00000ZGZCmUAP,9,7974,1,0,0,0,...,0,0,0,0,1,0,0,0,0,NaN
6,a062A00001VmW8yQAF,a062A00001VmW8yQAF,0012A000023XmOxQAK,a0I2A00000ZGZCnUAP,9,7974,1,0,0,0,...,0,0,0,0,1,0,0,0,0,NaN


In [18]:
tr_region.head()

,id,id_deals,id_accs,id_props,active_deals,activity_count,buyer_book,servicing_contract,cmbs,consultant,...,investor_type_Pension Fund,investor_type_Private Investor/Developer,investor_type_Property Lender,investor_type_Property Owner,investor_type_REIT,investor_type_Sovereign Wealth Fund,investor_type_Special Servicer,investor_type_Tenants in Common,investor_type_Unknown,region__c
0,a062A00001Whq5tQAB,a062A00001Whq5tQAB,0012A000023XlCCQA0,a0I2A00000XQKwXUAX,1,2367,1,0,0,0,...,0,1,0,0,0,0,0,0,0,Southeast
1,a062A00001VmP7EQAV,a062A00001VmP7EQAV,0012A000023XmOxQAK,a0I2A00000XvUEwUAN,9,7974,1,0,0,0,...,0,0,0,0,1,0,0,0,0,Southwest
8,a062A00001WhlUYQAZ,a062A00001WhlUYQAZ,0012A000023XmOxQAK,a0I2A00000XQHk4UAH,9,7974,1,0,0,0,...,0,0,0,0,1,0,0,0,0,Southwest
9,a062A00001Whq5sQAB,a062A00001Whq5sQAB,0012A000023XmOxQAK,a0I2A00000XQKwXUAX,9,7974,1,0,0,0,...,0,0,0,0,1,0,0,0,0,Southeast
10,a062A00001WiivKQAR,a062A00001WiivKQAR,0012A000023XmOxQAK,a0I2A00000XQLUWUA5,9,7974,1,0,0,0,...,0,0,0,0,1,0,0,0,0,Southwest


In [19]:
a_tr=tr_region['id_accs']
b_tr=tr_region['id_props']
a_te=te_region['id_accs']
b_te=te_region['id_props']
c_tr=tr_region['id_deals']
c_te=te_region['id_deals']
d_tr=tr_region['id']
d_te=te_region['id']

In [20]:
tr_region.drop(['id_accs','id_props'], axis=1, inplace=True)
te_region.drop(['id_accs','id_props'], axis=1, inplace=True)

In [21]:
tr_region.head()

,id,id_deals,active_deals,activity_count,buyer_book,servicing_contract,cmbs,consultant,correspondent,foreign,...,investor_type_Pension Fund,investor_type_Private Investor/Developer,investor_type_Property Lender,investor_type_Property Owner,investor_type_REIT,investor_type_Sovereign Wealth Fund,investor_type_Special Servicer,investor_type_Tenants in Common,investor_type_Unknown,region__c
0,a062A00001Whq5tQAB,a062A00001Whq5tQAB,1,2367,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,Southeast
1,a062A00001VmP7EQAV,a062A00001VmP7EQAV,9,7974,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,Southwest
8,a062A00001WhlUYQAZ,a062A00001WhlUYQAZ,9,7974,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,Southwest
9,a062A00001Whq5sQAB,a062A00001Whq5sQAB,9,7974,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,Southeast
10,a062A00001WiivKQAR,a062A00001WiivKQAR,9,7974,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,Southwest


In [22]:
region=tr_region['region__c']
tr_region.drop('region__c', axis=1, inplace=True)

In [23]:
te_region.drop('region__c', axis=1, inplace=True)

In [24]:
tr_region.drop('id', axis=1, inplace=True)
tr_region.drop('id_deals', axis=1, inplace=True)
te_region.drop('id', axis=1, inplace=True)
te_region.drop('id_deals', axis=1, inplace=True)

In [25]:
tr_region.head()

,active_deals,activity_count,buyer_book,servicing_contract,cmbs,consultant,correspondent,foreign,master_servicer,lender_book,...,investor_type_Life Insurance Company,investor_type_Pension Fund,investor_type_Private Investor/Developer,investor_type_Property Lender,investor_type_Property Owner,investor_type_REIT,investor_type_Sovereign Wealth Fund,investor_type_Special Servicer,investor_type_Tenants in Common,investor_type_Unknown
0,1,2367,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,9,7974,1,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
8,9,7974,1,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
9,9,7974,1,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
10,9,7974,1,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0


In [32]:
te_region.head()

,active_deals,activity_count,buyer_book,servicing_contract,cmbs,consultant,correspondent,foreign,master_servicer,lender_book,...,investor_type_Life Insurance Company,investor_type_Pension Fund,investor_type_Private Investor/Developer,investor_type_Property Lender,investor_type_Property Owner,investor_type_REIT,investor_type_Sovereign Wealth Fund,investor_type_Special Servicer,investor_type_Tenants in Common,investor_type_Unknown
2,9,7974,1,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
3,9,7974,1,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
4,9,7974,1,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
5,9,7974,1,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
6,9,7974,1,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0


In [34]:
from xgboost.sklearn import XGBClassifier
modelxgb_reg = XGBClassifier(
 learning_rate =0.1,
 n_estimators=1000,
 max_depth=5,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'multi:softprop',
 num_class=6, 
 nthread=4,
 scale_pos_weight=1,
 seed=27)
modelxgb_reg.fit(tr_region, region)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=1000,
       n_jobs=1, nthread=4, num_class=6, objective='multi:softprob',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=27, silent=True, subsample=0.8)

In [35]:
predict_reg=modelxgb_reg.predict(te_region)

C:\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [36]:
predict_reg.shape

(49491,)

In [37]:
te_region['region__c']=predict_reg

In [38]:
te_region.head()

,active_deals,activity_count,buyer_book,servicing_contract,cmbs,consultant,correspondent,foreign,master_servicer,lender_book,...,investor_type_Pension Fund,investor_type_Private Investor/Developer,investor_type_Property Lender,investor_type_Property Owner,investor_type_REIT,investor_type_Sovereign Wealth Fund,investor_type_Special Servicer,investor_type_Tenants in Common,investor_type_Unknown,region__c
2,9,7974,1,0,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,Southwest
3,9,7974,1,0,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,Southwest
4,9,7974,1,0,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,Southwest
5,9,7974,1,0,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,Southwest
6,9,7974,1,0,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,Southwest


In [45]:
tr_region.columns

Index(['Unnamed: 0', 'active_deals', 'activity_count', 'buyer_book',
       'servicing_contract', 'cmbs', 'consultant', 'correspondent', 'foreign',
       'master_servicer', 'lender_book', 'loan_sales_book', 'loan_servicing',
       'num_deals_as_client', 'num_deals_as_investor', 'number_of_properties',
       'number_of_related_deals', 'number_of_related_properties',
       'number_of_won_deals_as_client', 'investor_type_Account',
       'investor_type_Advisor/Equity Fund', 'investor_type_Agency',
       'investor_type_Bank / Thrift', 'investor_type_Conduit/CMBS',
       'investor_type_Corporate Investor/User',
       'investor_type_Credit/Specialty Finance Company',
       'investor_type_Endowment', 'investor_type_Life Insurance Company',
       'investor_type_Pension Fund',
       'investor_type_Private Investor/Developer',
       'investor_type_Property Lender', 'investor_type_Property Owner',
       'investor_type_REIT', 'investor_type_Sovereign Wealth Fund',
       'investor_type

In [39]:
tr_region['region__c']=region

In [41]:
tr_region.to_csv('tr_region_25.csv')

In [66]:
tr_region=pd.read_csv(r'tr_region.csv')

In [42]:
te_region.head()

,active_deals,activity_count,buyer_book,servicing_contract,cmbs,consultant,correspondent,foreign,master_servicer,lender_book,...,investor_type_Pension Fund,investor_type_Private Investor/Developer,investor_type_Property Lender,investor_type_Property Owner,investor_type_REIT,investor_type_Sovereign Wealth Fund,investor_type_Special Servicer,investor_type_Tenants in Common,investor_type_Unknown,region__c
2,9,7974,1,0,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,Southwest
3,9,7974,1,0,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,Southwest
4,9,7974,1,0,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,Southwest
5,9,7974,1,0,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,Southwest
6,9,7974,1,0,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,Southwest


In [69]:
tr_region.head()

,Unnamed: 0,active_deals,activity_count,buyer_book,servicing_contract,cmbs,consultant,correspondent,foreign,master_servicer,...,investor_type_Property Lender,investor_type_Property Owner,investor_type_REIT,investor_type_Sovereign Wealth Fund,investor_type_Special Servicer,investor_type_Tenants in Common,investor_type_Unknown,region__c,id_accs,id_props
0,0,1,2367,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,Southeast,0012A000023XlCCQA0,a0I2A00000XQKwXUAX
1,1,9,7974,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,Southwest,0012A000023XmOxQAK,a0I2A00000XvUEwUAN
2,8,9,7974,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,Southwest,0012A000023XmOxQAK,a0I2A00000XQHk4UAH
3,9,9,7974,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,Southeast,0012A000023XmOxQAK,a0I2A00000XQKwXUAX
4,10,9,7974,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,Southwest,0012A000023XmOxQAK,a0I2A00000XQLUWUA5


In [43]:
tr_region['id_accs']=a_tr
tr_region['id_props']=b_tr
tr_region['id_deals']=c_tr
tr_region['id']=d_tr

te_region['id_accs']=a_te
te_region['id_props']=b_te
te_region['id_deals']=c_te
te_region['id']=d_te

In [51]:
train_region=pd.concat([tr_region, te_region], axis=0)
train_region.shape

(70350, 41)

In [50]:
train_region.head()

,id,id_deals,id_accs,id_props,active_deals,activity_count,buyer_book,servicing_contract,cmbs,consultant,...,investor_type_Pension Fund,investor_type_Private Investor/Developer,investor_type_Property Lender,investor_type_Property Owner,investor_type_REIT,investor_type_Sovereign Wealth Fund,investor_type_Special Servicer,investor_type_Tenants in Common,investor_type_Unknown,region__c
0,a062A00001Whq5tQAB,a062A00001Whq5tQAB,0012A000023XlCCQA0,a0I2A00000XQKwXUAX,1,2367,1,0,0,0,...,0,1,0,0,0,0,0,0,0,Southeast
1,a062A00001VmP7EQAV,a062A00001VmP7EQAV,0012A000023XmOxQAK,a0I2A00000XvUEwUAN,9,7974,1,0,0,0,...,0,0,0,0,1,0,0,0,0,Southwest
8,a062A00001WhlUYQAZ,a062A00001WhlUYQAZ,0012A000023XmOxQAK,a0I2A00000XQHk4UAH,9,7974,1,0,0,0,...,0,0,0,0,1,0,0,0,0,Southwest
9,a062A00001Whq5sQAB,a062A00001Whq5sQAB,0012A000023XmOxQAK,a0I2A00000XQKwXUAX,9,7974,1,0,0,0,...,0,0,0,0,1,0,0,0,0,Southeast
10,a062A00001WiivKQAR,a062A00001WiivKQAR,0012A000023XmOxQAK,a0I2A00000XQLUWUA5,9,7974,1,0,0,0,...,0,0,0,0,1,0,0,0,0,Southwest


In [49]:
columnsTitles = ['id', 'id_deals', 'id_accs', 'id_props', 'active_deals',
       'activity_count', 'buyer_book', 'servicing_contract', 'cmbs',
       'consultant', 'correspondent', 'foreign', 'master_servicer',
       'lender_book', 'loan_sales_book', 'loan_servicing',
       'num_deals_as_client', 'num_deals_as_investor', 'number_of_properties',
       'number_of_related_deals', 'number_of_related_properties',
       'number_of_won_deals_as_client', 'investor_type_Account',
       'investor_type_Advisor/Equity Fund', 'investor_type_Agency',
       'investor_type_Bank / Thrift', 'investor_type_Conduit/CMBS',
       'investor_type_Corporate Investor/User',
       'investor_type_Credit/Specialty Finance Company',
       'investor_type_Endowment', 'investor_type_Life Insurance Company',
       'investor_type_Pension Fund',
       'investor_type_Private Investor/Developer',
       'investor_type_Property Lender', 'investor_type_Property Owner',
       'investor_type_REIT', 'investor_type_Sovereign Wealth Fund',
       'investor_type_Special Servicer', 'investor_type_Tenants in Common',
       'investor_type_Unknown', 'region__c']

train_region=train_region.reindex(columns=columnsTitles)

In [52]:
train_region.to_csv('train_region_25.csv')

In [53]:
train_region.shape

(70350, 41)

In [54]:
train_region.head()

,id,id_deals,id_accs,id_props,active_deals,activity_count,buyer_book,servicing_contract,cmbs,consultant,...,investor_type_Pension Fund,investor_type_Private Investor/Developer,investor_type_Property Lender,investor_type_Property Owner,investor_type_REIT,investor_type_Sovereign Wealth Fund,investor_type_Special Servicer,investor_type_Tenants in Common,investor_type_Unknown,region__c
0,a062A00001Whq5tQAB,a062A00001Whq5tQAB,0012A000023XlCCQA0,a0I2A00000XQKwXUAX,1,2367,1,0,0,0,...,0,1,0,0,0,0,0,0,0,Southeast
1,a062A00001VmP7EQAV,a062A00001VmP7EQAV,0012A000023XmOxQAK,a0I2A00000XvUEwUAN,9,7974,1,0,0,0,...,0,0,0,0,1,0,0,0,0,Southwest
8,a062A00001WhlUYQAZ,a062A00001WhlUYQAZ,0012A000023XmOxQAK,a0I2A00000XQHk4UAH,9,7974,1,0,0,0,...,0,0,0,0,1,0,0,0,0,Southwest
9,a062A00001Whq5sQAB,a062A00001Whq5sQAB,0012A000023XmOxQAK,a0I2A00000XQKwXUAX,9,7974,1,0,0,0,...,0,0,0,0,1,0,0,0,0,Southeast
10,a062A00001WiivKQAR,a062A00001WiivKQAR,0012A000023XmOxQAK,a0I2A00000XQLUWUA5,9,7974,1,0,0,0,...,0,0,0,0,1,0,0,0,0,Southwest


In [ ]:
#####################################################SALE STATUS BEGIN

In [55]:
ap_acc_prop.drop('region__c', axis=1, inplace=True)

In [56]:
ap_acc_prop.head()

,id,id_deals,id_accs,id_props,active_deals,activity_count,buyer_book,servicing_contract,cmbs,consultant,...,investor_type_Life Insurance Company,investor_type_Pension Fund,investor_type_Private Investor/Developer,investor_type_Property Lender,investor_type_Property Owner,investor_type_REIT,investor_type_Sovereign Wealth Fund,investor_type_Special Servicer,investor_type_Tenants in Common,investor_type_Unknown
0,a062A00001Whq5tQAB,a062A00001Whq5tQAB,0012A000023XlCCQA0,a0I2A00000XQKwXUAX,1,2367,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,a062A00001VmP7EQAV,a062A00001VmP7EQAV,0012A000023XmOxQAK,a0I2A00000XvUEwUAN,9,7974,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,a062A00001VmQByQAN,a062A00001VmQByQAN,0012A000023XmOxQAK,a0I2A00000XwQ13UAF,9,7974,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,a062A00001VmQC2QAN,a062A00001VmQC2QAN,0012A000023XmOxQAK,a0I2A00000XwZyBUAV,9,7974,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,a062A00001VmW8wQAF,a062A00001VmW8wQAF,0012A000023XmOxQAK,a0I2A00000ZGZClUAP,9,7974,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [67]:
ap_acc_prop=ap_acc.merge(prop[['id_props','sale_status']], on=['id_props'],how='left' )

In [68]:
te_ss=pd.DataFrame(ap_acc_prop[ap_acc_prop['sale_status'].isnull()])
tr_ss=pd.DataFrame(ap_acc_prop[ap_acc_prop['sale_status'].notnull()])

In [69]:
a_tr=tr_region['id_accs']
b_tr=tr_region['id_props']
c_tr=tr_region['id_deals']
d_tr=tr_region['id']

a_te=te_region['id_accs']
b_te=te_region['id_props']
c_te=te_region['id_deals']
d_te=te_region['id']

In [72]:
sale_status=tr_ss['sale_status']

tr_ss.drop(['id_accs','id_props'], axis=1, inplace=True)
tr_ss.drop('sale_status', axis=1, inplace=True)
tr_ss.drop('id', axis=1, inplace=True)
tr_ss.drop('id_deals', axis=1, inplace=True)

te_ss.drop(['id_accs','id_props'], axis=1, inplace=True)
te_ss.drop('sale_status', axis=1, inplace=True)
te_ss.drop('id', axis=1, inplace=True)
te_ss.drop('id_deals', axis=1, inplace=True)

In [73]:
te_ss.head()

,active_deals,activity_count,buyer_book,servicing_contract,cmbs,consultant,correspondent,foreign,master_servicer,lender_book,...,investor_type_Life Insurance Company,investor_type_Pension Fund,investor_type_Private Investor/Developer,investor_type_Property Lender,investor_type_Property Owner,investor_type_REIT,investor_type_Sovereign Wealth Fund,investor_type_Special Servicer,investor_type_Tenants in Common,investor_type_Unknown
2,9,7974,1,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
3,9,7974,1,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
4,9,7974,1,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
5,9,7974,1,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
6,9,7974,1,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0


In [74]:
te_ss.head()

,active_deals,activity_count,buyer_book,servicing_contract,cmbs,consultant,correspondent,foreign,master_servicer,lender_book,...,investor_type_Life Insurance Company,investor_type_Pension Fund,investor_type_Private Investor/Developer,investor_type_Property Lender,investor_type_Property Owner,investor_type_REIT,investor_type_Sovereign Wealth Fund,investor_type_Special Servicer,investor_type_Tenants in Common,investor_type_Unknown
2,9,7974,1,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
3,9,7974,1,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
4,9,7974,1,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
5,9,7974,1,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
6,9,7974,1,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0


In [77]:
from xgboost.sklearn import XGBClassifier
modelxgb_ss = XGBClassifier(
 learning_rate =0.1,
 n_estimators=1000,
 max_depth=5,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
 seed=27)
modelxgb_ss.fit(tr_ss, sale_status)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=1000,
       n_jobs=1, nthread=4, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=27, silent=True,
       subsample=0.8)

In [78]:
predict_ss=modelxgb_ss.predict(te_ss)

C:\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [93]:
te_ss['sale_status']=predict_ss
te_ss.head()

,active_deals,activity_count,buyer_book,servicing_contract,cmbs,consultant,correspondent,foreign,master_servicer,lender_book,...,investor_type_REIT,investor_type_Sovereign Wealth Fund,investor_type_Special Servicer,investor_type_Tenants in Common,investor_type_Unknown,sale_status,id_accs,id_props,id_deals,id
2,9,7974,1,0,0,0,0,0,0,1,...,1,0,0,0,0,N,0012A000023XmOxQAK,a0I2A00000XwQ13UAF,a062A00001VmQByQAN,a062A00001VmQByQAN
3,9,7974,1,0,0,0,0,0,0,1,...,1,0,0,0,0,N,0012A000023XmOxQAK,a0I2A00000XwZyBUAV,a062A00001VmQC2QAN,a062A00001VmQC2QAN
4,9,7974,1,0,0,0,0,0,0,1,...,1,0,0,0,0,N,0012A000023XmOxQAK,a0I2A00000ZGZClUAP,a062A00001VmW8wQAF,a062A00001VmW8wQAF
5,9,7974,1,0,0,0,0,0,0,1,...,1,0,0,0,0,N,0012A000023XmOxQAK,a0I2A00000ZGZCmUAP,a062A00001VmW8xQAF,a062A00001VmW8xQAF
6,9,7974,1,0,0,0,0,0,0,1,...,1,0,0,0,0,N,0012A000023XmOxQAK,a0I2A00000ZGZCnUAP,a062A00001VmW8yQAF,a062A00001VmW8yQAF


In [99]:
tr_ss=pd.DataFrame(ap_acc_prop[ap_acc_prop['sale_status'].notnull()])
tr_ss.head(10)

,id,id_deals,id_accs,id_props,active_deals,activity_count,buyer_book,servicing_contract,cmbs,consultant,...,investor_type_Pension Fund,investor_type_Private Investor/Developer,investor_type_Property Lender,investor_type_Property Owner,investor_type_REIT,investor_type_Sovereign Wealth Fund,investor_type_Special Servicer,investor_type_Tenants in Common,investor_type_Unknown,sale_status
0,a062A00001Whq5tQAB,a062A00001Whq5tQAB,0012A000023XlCCQA0,a0I2A00000XQKwXUAX,1,2367,1,0,0,0,...,0,1,0,0,0,0,0,0,0,N
1,a062A00001VmP7EQAV,a062A00001VmP7EQAV,0012A000023XmOxQAK,a0I2A00000XvUEwUAN,9,7974,1,0,0,0,...,0,0,0,0,1,0,0,0,0,N
8,a062A00001WhlUYQAZ,a062A00001WhlUYQAZ,0012A000023XmOxQAK,a0I2A00000XQHk4UAH,9,7974,1,0,0,0,...,0,0,0,0,1,0,0,0,0,N
9,a062A00001Whq5sQAB,a062A00001Whq5sQAB,0012A000023XmOxQAK,a0I2A00000XQKwXUAX,9,7974,1,0,0,0,...,0,0,0,0,1,0,0,0,0,N
10,a062A00001WiivKQAR,a062A00001WiivKQAR,0012A000023XmOxQAK,a0I2A00000XQLUWUA5,9,7974,1,0,0,0,...,0,0,0,0,1,0,0,0,0,N
30,a062A00001Wj5B6QAJ,a062A00001Wj5B6QAJ,0012A000023XmOxQAK,a0I2A00000XQGxsUAH,9,7974,1,0,0,0,...,0,0,0,0,1,0,0,0,0,N
86,a062A00001VluvmQAB,a062A00001VluvmQAB,0012A000023YLqwQAG,a0I2A00000XQIHiUAP,891,50161,1,1,0,0,...,0,0,0,0,0,0,0,0,0,N
87,a062A00001VlwEsQAJ,a062A00001VlwEsQAJ,0012A000023YLqwQAG,a0I2A00000XQVacUAH,891,50161,1,1,0,0,...,0,0,0,0,0,0,0,0,0,N
88,a062A00001VlwF1QAJ,a062A00001VlwF1QAJ,0012A000023YLqwQAG,a0I2A00000XwOPFUA3,891,50161,1,1,0,0,...,0,0,0,0,0,0,0,0,0,N
89,a062A00001VlwF2QAJ,a062A00001VlwF2QAJ,0012A000023YLqwQAG,a0I2A00000XweIHUAZ,891,50161,1,1,0,0,...,0,0,0,0,0,0,0,0,0,N


In [101]:
te_ss.head()

,id,id_deals,id_accs,id_props,active_deals,activity_count,buyer_book,servicing_contract,cmbs,consultant,...,investor_type_Pension Fund,investor_type_Private Investor/Developer,investor_type_Property Lender,investor_type_Property Owner,investor_type_REIT,investor_type_Sovereign Wealth Fund,investor_type_Special Servicer,investor_type_Tenants in Common,investor_type_Unknown,sale_status
2,a062A00001VmQByQAN,a062A00001VmQByQAN,0012A000023XmOxQAK,a0I2A00000XwQ13UAF,9,7974,1,0,0,0,...,0,0,0,0,1,0,0,0,0,N
3,a062A00001VmQC2QAN,a062A00001VmQC2QAN,0012A000023XmOxQAK,a0I2A00000XwZyBUAV,9,7974,1,0,0,0,...,0,0,0,0,1,0,0,0,0,N
4,a062A00001VmW8wQAF,a062A00001VmW8wQAF,0012A000023XmOxQAK,a0I2A00000ZGZClUAP,9,7974,1,0,0,0,...,0,0,0,0,1,0,0,0,0,N
5,a062A00001VmW8xQAF,a062A00001VmW8xQAF,0012A000023XmOxQAK,a0I2A00000ZGZCmUAP,9,7974,1,0,0,0,...,0,0,0,0,1,0,0,0,0,N
6,a062A00001VmW8yQAF,a062A00001VmW8yQAF,0012A000023XmOxQAK,a0I2A00000ZGZCnUAP,9,7974,1,0,0,0,...,0,0,0,0,1,0,0,0,0,N


In [83]:
tr_ss['id_accs']=a_tr
tr_ss['id_props']=b_tr
tr_ss['id_deals']=c_tr
tr_ss['id']=d_tr

te_ss['id_accs']=a_te
te_ss['id_props']=b_te
te_ss['id_deals']=c_te
te_ss['id']=d_te

In [102]:
train_ss=pd.concat([tr_ss, te_ss], axis=0)
train_ss.shape

(70350, 41)

In [88]:
columnsTitles = ['id', 'id_deals', 'id_accs', 'id_props', 'active_deals',
       'activity_count', 'buyer_book', 'servicing_contract', 'cmbs',
       'consultant', 'correspondent', 'foreign', 'master_servicer',
       'lender_book', 'loan_sales_book', 'loan_servicing',
       'num_deals_as_client', 'num_deals_as_investor', 'number_of_properties',
       'number_of_related_deals', 'number_of_related_properties',
       'number_of_won_deals_as_client', 'investor_type_Account',
       'investor_type_Advisor/Equity Fund', 'investor_type_Agency',
       'investor_type_Bank / Thrift', 'investor_type_Conduit/CMBS',
       'investor_type_Corporate Investor/User',
       'investor_type_Credit/Specialty Finance Company',
       'investor_type_Endowment', 'investor_type_Life Insurance Company',
       'investor_type_Pension Fund',
       'investor_type_Private Investor/Developer',
       'investor_type_Property Lender', 'investor_type_Property Owner',
       'investor_type_REIT', 'investor_type_Sovereign Wealth Fund',
       'investor_type_Special Servicer', 'investor_type_Tenants in Common',
       'investor_type_Unknown', 'sale_status']

train_ss=train_ss.reindex(columns=columnsTitles)

In [103]:
train_ss.head()

,id,id_deals,id_accs,id_props,active_deals,activity_count,buyer_book,servicing_contract,cmbs,consultant,...,investor_type_Pension Fund,investor_type_Private Investor/Developer,investor_type_Property Lender,investor_type_Property Owner,investor_type_REIT,investor_type_Sovereign Wealth Fund,investor_type_Special Servicer,investor_type_Tenants in Common,investor_type_Unknown,sale_status
0,a062A00001Whq5tQAB,a062A00001Whq5tQAB,0012A000023XlCCQA0,a0I2A00000XQKwXUAX,1,2367,1,0,0,0,...,0,1,0,0,0,0,0,0,0,N
1,a062A00001VmP7EQAV,a062A00001VmP7EQAV,0012A000023XmOxQAK,a0I2A00000XvUEwUAN,9,7974,1,0,0,0,...,0,0,0,0,1,0,0,0,0,N
8,a062A00001WhlUYQAZ,a062A00001WhlUYQAZ,0012A000023XmOxQAK,a0I2A00000XQHk4UAH,9,7974,1,0,0,0,...,0,0,0,0,1,0,0,0,0,N
9,a062A00001Whq5sQAB,a062A00001Whq5sQAB,0012A000023XmOxQAK,a0I2A00000XQKwXUAX,9,7974,1,0,0,0,...,0,0,0,0,1,0,0,0,0,N
10,a062A00001WiivKQAR,a062A00001WiivKQAR,0012A000023XmOxQAK,a0I2A00000XQLUWUA5,9,7974,1,0,0,0,...,0,0,0,0,1,0,0,0,0,N


In [104]:
train_ss['sale_status'].unique()

array(['N', 'Y'], dtype=object)

In [ ]:
##################################################################PORTFOLIO BEGIN

In [106]:
ap_acc_prop.drop('sale_status', axis=1, inplace=True)

In [107]:
prop['portfolio'].unique()

array([False,  True])

In [125]:
ap_acc_prop=ap_acc.merge(prop[['id_props','portfolio']], on=['id_props'],how='left' )
te_portfolio=pd.DataFrame(ap_acc_prop[ap_acc_prop['portfolio'].isnull()])
tr_portfolio=pd.DataFrame(ap_acc_prop[ap_acc_prop['portfolio'].notnull()])

a_tr=tr_portfolio['id_accs']
b_tr=tr_portfolio['id_props']
c_tr=tr_portfolio['id_deals']
d_tr=tr_portfolio['id']

a_te=te_portfolio['id_accs']
b_te=te_portfolio['id_props']
c_te=te_portfolio['id_deals']
d_te=te_portfolio['id']

portfolio=tr_portfolio['portfolio']

tr_portfolio.drop(['id_accs','id_props'], axis=1, inplace=True)
tr_portfolio.drop('portfolio', axis=1, inplace=True)
tr_portfolio.drop('id', axis=1, inplace=True)
tr_portfolio.drop('id_deals', axis=1, inplace=True)

te_portfolio.drop(['id_accs','id_props'], axis=1, inplace=True)
te_portfolio.drop('portfolio', axis=1, inplace=True)
te_portfolio.drop('id', axis=1, inplace=True)
te_portfolio.drop('id_deals', axis=1, inplace=True)


In [126]:
tr_portfolio.head()

,active_deals,activity_count,buyer_book,servicing_contract,cmbs,consultant,correspondent,foreign,master_servicer,lender_book,...,investor_type_Life Insurance Company,investor_type_Pension Fund,investor_type_Private Investor/Developer,investor_type_Property Lender,investor_type_Property Owner,investor_type_REIT,investor_type_Sovereign Wealth Fund,investor_type_Special Servicer,investor_type_Tenants in Common,investor_type_Unknown
0,1,2367,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,9,7974,1,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
8,9,7974,1,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
9,9,7974,1,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
10,9,7974,1,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0


In [127]:
te_portfolio.head()

,active_deals,activity_count,buyer_book,servicing_contract,cmbs,consultant,correspondent,foreign,master_servicer,lender_book,...,investor_type_Life Insurance Company,investor_type_Pension Fund,investor_type_Private Investor/Developer,investor_type_Property Lender,investor_type_Property Owner,investor_type_REIT,investor_type_Sovereign Wealth Fund,investor_type_Special Servicer,investor_type_Tenants in Common,investor_type_Unknown
2,9,7974,1,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
3,9,7974,1,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
4,9,7974,1,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
5,9,7974,1,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
6,9,7974,1,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0


In [128]:
portfolio=tr_portfolio['portfolio']

tr_portfolio.drop(['id_accs','id_props'], axis=1, inplace=True)
tr_portfolio.drop('portfolio', axis=1, inplace=True)
tr_portfolio.drop('id', axis=1, inplace=True)
tr_portfolio.drop('id_deals', axis=1, inplace=True)

te_portfolio.drop(['id_accs','id_props'], axis=1, inplace=True)
te_portfolio.drop('portfolio', axis=1, inplace=True)
te_portfolio.drop('id', axis=1, inplace=True)
te_portfolio.drop('id_deals', axis=1, inplace=True)

from xgboost.sklearn import XGBClassifier
modelxgb_port = XGBClassifier(
 learning_rate =0.1,
 n_estimators=1000,
 max_depth=5,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
 seed=27)
modelxgb_port.fit(tr_portfolio, portfolio)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=1000,
       n_jobs=1, nthread=4, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=27, silent=True,
       subsample=0.8)

In [129]:
predict_portfolio=modelxgb_port.predict(te_portfolio)

C:\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [ ]:
tr_portfolio['id_accs']=a_tr
tr_portfolio['id_props']=b_tr
tr_portfolio['id_deals']=c_tr
tr_portfolio['id']=d_tr

te_portfolio['id_accs']=a_te
te_portfolio['id_props']=b_te
te_portfolio['id_deals']=c_te
te_portfolio['id']=d_te

In [134]:
te_portfolio['portfolio']=predict_portfolio
columnsTitles = ['id', 'id_deals', 'id_accs', 'id_props', 'active_deals',
       'activity_count', 'buyer_book', 'servicing_contract', 'cmbs',
       'consultant', 'correspondent', 'foreign', 'master_servicer',
       'lender_book', 'loan_sales_book', 'loan_servicing',
       'num_deals_as_client', 'num_deals_as_investor', 'number_of_properties',
       'number_of_related_deals', 'number_of_related_properties',
       'number_of_won_deals_as_client', 'investor_type_Account',
       'investor_type_Advisor/Equity Fund', 'investor_type_Agency',
       'investor_type_Bank / Thrift', 'investor_type_Conduit/CMBS',
       'investor_type_Corporate Investor/User',
       'investor_type_Credit/Specialty Finance Company',
       'investor_type_Endowment', 'investor_type_Life Insurance Company',
       'investor_type_Pension Fund',
       'investor_type_Private Investor/Developer',
       'investor_type_Property Lender', 'investor_type_Property Owner',
       'investor_type_REIT', 'investor_type_Sovereign Wealth Fund',
       'investor_type_Special Servicer', 'investor_type_Tenants in Common',
       'investor_type_Unknown', 'portfolio']

te_portfolio=te_portfolio.reindex(columns=columnsTitles)
te_portfolio.head()

,id,id_deals,id_accs,id_props,active_deals,activity_count,buyer_book,servicing_contract,cmbs,consultant,...,investor_type_Pension Fund,investor_type_Private Investor/Developer,investor_type_Property Lender,investor_type_Property Owner,investor_type_REIT,investor_type_Sovereign Wealth Fund,investor_type_Special Servicer,investor_type_Tenants in Common,investor_type_Unknown,portfolio
2,a062A00001VmQByQAN,a062A00001VmQByQAN,0012A000023XmOxQAK,a0I2A00000XwQ13UAF,9,7974,1,0,0,0,...,0,0,0,0,1,0,0,0,0,False
3,a062A00001VmQC2QAN,a062A00001VmQC2QAN,0012A000023XmOxQAK,a0I2A00000XwZyBUAV,9,7974,1,0,0,0,...,0,0,0,0,1,0,0,0,0,False
4,a062A00001VmW8wQAF,a062A00001VmW8wQAF,0012A000023XmOxQAK,a0I2A00000ZGZClUAP,9,7974,1,0,0,0,...,0,0,0,0,1,0,0,0,0,False
5,a062A00001VmW8xQAF,a062A00001VmW8xQAF,0012A000023XmOxQAK,a0I2A00000ZGZCmUAP,9,7974,1,0,0,0,...,0,0,0,0,1,0,0,0,0,False
6,a062A00001VmW8yQAF,a062A00001VmW8yQAF,0012A000023XmOxQAK,a0I2A00000ZGZCnUAP,9,7974,1,0,0,0,...,0,0,0,0,1,0,0,0,0,False


In [135]:
tr_portfolio['portfolio']=portfolio
columnsTitles = ['id', 'id_deals', 'id_accs', 'id_props', 'active_deals',
       'activity_count', 'buyer_book', 'servicing_contract', 'cmbs',
       'consultant', 'correspondent', 'foreign', 'master_servicer',
       'lender_book', 'loan_sales_book', 'loan_servicing',
       'num_deals_as_client', 'num_deals_as_investor', 'number_of_properties',
       'number_of_related_deals', 'number_of_related_properties',
       'number_of_won_deals_as_client', 'investor_type_Account',
       'investor_type_Advisor/Equity Fund', 'investor_type_Agency',
       'investor_type_Bank / Thrift', 'investor_type_Conduit/CMBS',
       'investor_type_Corporate Investor/User',
       'investor_type_Credit/Specialty Finance Company',
       'investor_type_Endowment', 'investor_type_Life Insurance Company',
       'investor_type_Pension Fund',
       'investor_type_Private Investor/Developer',
       'investor_type_Property Lender', 'investor_type_Property Owner',
       'investor_type_REIT', 'investor_type_Sovereign Wealth Fund',
       'investor_type_Special Servicer', 'investor_type_Tenants in Common',
       'investor_type_Unknown', 'portfolio']

tr_portfolio=tr_portfolio.reindex(columns=columnsTitles)

tr_portfolio.head()

,id,id_deals,id_accs,id_props,active_deals,activity_count,buyer_book,servicing_contract,cmbs,consultant,...,investor_type_Pension Fund,investor_type_Private Investor/Developer,investor_type_Property Lender,investor_type_Property Owner,investor_type_REIT,investor_type_Sovereign Wealth Fund,investor_type_Special Servicer,investor_type_Tenants in Common,investor_type_Unknown,portfolio
0,a062A00001Whq5tQAB,a062A00001Whq5tQAB,0012A000023XlCCQA0,a0I2A00000XQKwXUAX,1,2367,1,0,0,0,...,0,1,0,0,0,0,0,0,0,False
1,a062A00001VmP7EQAV,a062A00001VmP7EQAV,0012A000023XmOxQAK,a0I2A00000XvUEwUAN,9,7974,1,0,0,0,...,0,0,0,0,1,0,0,0,0,False
8,a062A00001WhlUYQAZ,a062A00001WhlUYQAZ,0012A000023XmOxQAK,a0I2A00000XQHk4UAH,9,7974,1,0,0,0,...,0,0,0,0,1,0,0,0,0,False
9,a062A00001Whq5sQAB,a062A00001Whq5sQAB,0012A000023XmOxQAK,a0I2A00000XQKwXUAX,9,7974,1,0,0,0,...,0,0,0,0,1,0,0,0,0,False
10,a062A00001WiivKQAR,a062A00001WiivKQAR,0012A000023XmOxQAK,a0I2A00000XQLUWUA5,9,7974,1,0,0,0,...,0,0,0,0,1,0,0,0,0,False


In [138]:
train_portfolio=pd.concat([tr_portfolio, te_portfolio], axis=0)
train_portfolio.shape

(70350, 41)

In [139]:
train_portfolio.head()

,id,id_deals,id_accs,id_props,active_deals,activity_count,buyer_book,servicing_contract,cmbs,consultant,...,investor_type_Pension Fund,investor_type_Private Investor/Developer,investor_type_Property Lender,investor_type_Property Owner,investor_type_REIT,investor_type_Sovereign Wealth Fund,investor_type_Special Servicer,investor_type_Tenants in Common,investor_type_Unknown,portfolio
0,a062A00001Whq5tQAB,a062A00001Whq5tQAB,0012A000023XlCCQA0,a0I2A00000XQKwXUAX,1,2367,1,0,0,0,...,0,1,0,0,0,0,0,0,0,False
1,a062A00001VmP7EQAV,a062A00001VmP7EQAV,0012A000023XmOxQAK,a0I2A00000XvUEwUAN,9,7974,1,0,0,0,...,0,0,0,0,1,0,0,0,0,False
8,a062A00001WhlUYQAZ,a062A00001WhlUYQAZ,0012A000023XmOxQAK,a0I2A00000XQHk4UAH,9,7974,1,0,0,0,...,0,0,0,0,1,0,0,0,0,False
9,a062A00001Whq5sQAB,a062A00001Whq5sQAB,0012A000023XmOxQAK,a0I2A00000XQKwXUAX,9,7974,1,0,0,0,...,0,0,0,0,1,0,0,0,0,False
10,a062A00001WiivKQAR,a062A00001WiivKQAR,0012A000023XmOxQAK,a0I2A00000XQLUWUA5,9,7974,1,0,0,0,...,0,0,0,0,1,0,0,0,0,False


In [140]:
train_portfolio['portfolio'].unique()

array([False, True], dtype=object)

In [141]:
train_portfolio.to_csv('train_portfolio_25.csv')

In [167]:
trainfinal_25 = pd.read_csv(r'C:\Users\Vincy Sagar\Desktop\Affine\train_final_25 - Copy.csv')

C:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [168]:
trainfinal_25.head()

,id,id_deals,id_accs,id_props,sale_status,region__c,portfolio,active_deals,activity_count,buyer_book,...,investor_type_Life Insurance Company,investor_type_Pension Fund,investor_type_Private Investor/Developer,investor_type_Property Lender,investor_type_Property Owner,investor_type_REIT,investor_type_Sovereign Wealth Fund,investor_type_Special Servicer,investor_type_Tenants in Common,investor_type_Unknown
0,a062A00001Whq5tQAB,a062A00001Whq5tQAB,0012A000023XlCCQA0,a0I2A00000XQKwXUAX,N,Southeast,False,1,2367,1,...,0,0,1,0,0,0,0,0,0,0
1,a062A00001VmP7EQAV,a062A00001VmP7EQAV,0012A000023XmOxQAK,a0I2A00000XvUEwUAN,N,Southwest,False,9,7974,1,...,0,0,0,0,0,1,0,0,0,0
2,a062A00001WhlUYQAZ,a062A00001WhlUYQAZ,0012A000023XmOxQAK,a0I2A00000XQHk4UAH,N,Southwest,False,9,7974,1,...,0,0,0,0,0,1,0,0,0,0
3,a062A00001Whq5sQAB,a062A00001Whq5sQAB,0012A000023XmOxQAK,a0I2A00000XQKwXUAX,N,Southeast,False,9,7974,1,...,0,0,0,0,0,1,0,0,0,0
4,a062A00001WiivKQAR,a062A00001WiivKQAR,0012A000023XmOxQAK,a0I2A00000XQLUWUA5,N,Southwest,False,9,7974,1,...,0,0,0,0,0,1,0,0,0,0


In [169]:
trainfinal_25_orig = trainfinal_25.copy()

In [170]:
trainfinal_25.drop(['id','id_deals','id_accs'],axis=1,inplace=True)

In [185]:
test = test.merge(trainfinal_25[['id_props','sale_status','region__c','portfolio','investor_type_Private Investor/Developer']] , on=['sale_status','region__c','portfolio','investor_type_Private Investor/Developer'], how = 'left')

In [171]:
test = test.merge(trainfinal_25[['id_props','sale_status','region__c','portfolio','buyer_book' , 'servicing_contract' , 'cmbs' , 'consultant' ,'correspondent' , 'foreign' , 'master_servicer' , 'lender_book'  , 'loan_sales_book' , 'loan_servicing','investor_type_Account',
       'investor_type_Advisor/Equity Fund', 'investor_type_Agency',
       'investor_type_Bank / Thrift', 'investor_type_Conduit/CMBS',
       'investor_type_Corporate Investor/User',
       'investor_type_Credit/Specialty Finance Company',
       'investor_type_Endowment', 'investor_type_Life Insurance Company',
       'investor_type_Pension Fund',
       'investor_type_Private Investor/Developer',
       'investor_type_Property Lender', 'investor_type_Property Owner',
       'investor_type_REIT', 'investor_type_Sovereign Wealth Fund',
       'investor_type_Special Servicer', 'investor_type_Tenants in Common',
       'investor_type_Unknown']] , on=['sale_status','region__c','portfolio','buyer_book' , 'servicing_contract' , 'cmbs' , 'consultant' ,'correspondent' , 'foreign' , 'master_servicer' , 'lender_book'  , 'loan_sales_book' , 'loan_servicing','investor_type_Account',
       'investor_type_Advisor/Equity Fund', 'investor_type_Agency',
       'investor_type_Bank / Thrift', 'investor_type_Conduit/CMBS',
       'investor_type_Corporate Investor/User',
       'investor_type_Credit/Specialty Finance Company',
       'investor_type_Endowment', 'investor_type_Life Insurance Company',
       'investor_type_Pension Fund',
       'investor_type_Private Investor/Developer',
       'investor_type_Property Lender', 'investor_type_Property Owner',
       'investor_type_REIT', 'investor_type_Sovereign Wealth Fund',
       'investor_type_Special Servicer', 'investor_type_Tenants in Common',
       'investor_type_Unknown'], how = 'left')

In [186]:
test.head(10)

,id_accs,active_deals,activity_count,sale_status,region__c,portfolio,buyer_book,servicing_contract,cmbs,consultant,...,investor_type_Pension Fund,investor_type_Private Investor/Developer,investor_type_Property Lender,investor_type_Property Owner,investor_type_REIT,investor_type_Sovereign Wealth Fund,investor_type_Special Servicer,investor_type_Tenants in Common,investor_type_Unknown,id_props
0,0012A00002AXHJUQA5,113,203365,N,Southeast,False,1,0,0,0,...,0,0,0,0,0,0,0,0,0,a0I2A00000XQKwXUAX
1,0012A00002AXHJUQA5,113,203365,N,Southeast,False,1,0,0,0,...,0,0,0,0,0,0,0,0,0,a0I2A00000XQGxsUAH
2,0012A00002AXHJUQA5,113,203365,N,Southeast,False,1,0,0,0,...,0,0,0,0,0,0,0,0,0,a0I2A00000XQIT1UAP
3,0012A00002AXHJUQA5,113,203365,N,Southeast,False,1,0,0,0,...,0,0,0,0,0,0,0,0,0,a0I2A00000XQGLrUAP
4,0012A00002AXHJUQA5,113,203365,N,Southeast,False,1,0,0,0,...,0,0,0,0,0,0,0,0,0,a0I2A00000XQGntUAH
5,0012A00002AXHJUQA5,113,203365,N,Southeast,False,1,0,0,0,...,0,0,0,0,0,0,0,0,0,a0I2A00000XQH7hUAH
6,0012A00002AXHJUQA5,113,203365,N,Southeast,False,1,0,0,0,...,0,0,0,0,0,0,0,0,0,a0I2A00000XQHG1UAP
7,0012A00002AXHJUQA5,113,203365,N,Southeast,False,1,0,0,0,...,0,0,0,0,0,0,0,0,0,a0I2A00000XQH7DUAX
8,0012A00002AXHJUQA5,113,203365,N,Southeast,False,1,0,0,0,...,0,0,0,0,0,0,0,0,0,a0I2A00000XQHZnUAP
9,0012A00002AXHJUQA5,113,203365,N,Southeast,False,1,0,0,0,...,0,0,0,0,0,0,0,0,0,a0I2A00000XQHcjUAH


In [146]:
test = test.merge(acc, on=['id_accs'], how='left')

In [163]:
#test.to_csv('test25.csv')

In [187]:
testsub25 = pd.DataFrame({'id_accs':test['id_accs'],'id_prop':test['id_props']})

In [182]:
testsub25.head()

,id_accs,id_prop
0,0012A00002AXHJUQA5,a0I2A00000XQKwXUAX
1,0012A00002AXHJUQA5,a0I2A00000XQKwXUAX
2,0012A00002AXHJUQA5,a0I2A00000XQGxsUAH
3,0012A00002AXHJUQA5,a0I2A00000XQIT1UAP
4,0012A00002AXHJUQA5,a0I2A00000XQGLrUAP


In [188]:
testsub25.to_csv('testsub25_2.csv')

In [53]:
test.shape

(29, 1)

In [109]:
test_region = test.merge(acc, on=['id_accs'], how='left')

In [110]:
test_region.shape

(29, 37)

In [111]:
test_region.head()

,id_accs,active_deals,activity_count,buyer_book,servicing_contract,cmbs,consultant,correspondent,foreign,master_servicer,...,investor_type_Life Insurance Company,investor_type_Pension Fund,investor_type_Private Investor/Developer,investor_type_Property Lender,investor_type_Property Owner,investor_type_REIT,investor_type_Sovereign Wealth Fund,investor_type_Special Servicer,investor_type_Tenants in Common,investor_type_Unknown
0,0012A00002AXHJUQA5,113,203365,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0012A000023XlWaQAK,261,127948,1,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,0012A000023Y4bYQAS,379,204360,1,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3,0012A000023XlbuQAC,284,132408,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0012A000023Xlh9QAC,299,11365,1,1,0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,0


In [114]:
test_region.shape

(29, 36)

In [79]:
idacc=train_region['id_accs']
ipprop=train_region['id_props']

In [80]:
train_region.drop(['id_accs','id_props'], axis=1, inplace=True)
test_region.drop(['id_accs'], axis=1, inplace=True)

In [112]:
test_region.drop(['id_accs'], axis=1, inplace=True)

In [81]:
regn=train_region['region__c']
train_region.drop(['region__c'], axis=1, inplace=True)

In [60]:
regn.describe()

count         20859
unique            6
top       Southwest
freq           6306
Name: region__c, dtype: object

In [113]:
train_region.head()

,Unnamed: 0,active_deals,activity_count,buyer_book,cmbs,consultant,correspondent,foreign,investor_type_Account,investor_type_Advisor/Equity Fund,...,loan_sales_book,loan_servicing,master_servicer,num_deals_as_client,num_deals_as_investor,number_of_properties,number_of_related_deals,number_of_related_properties,number_of_won_deals_as_client,servicing_contract
0,0.0,1,2367,1,0,0,0,0,0,0,...,0,0,0,0,1,1,2,1,0,0
1,1.0,9,7974,1,0,0,0,0,0,0,...,1,0,0,7,2,77,18,85,7,0
2,8.0,9,7974,1,0,0,0,0,0,0,...,1,0,0,7,2,77,18,85,7,0
3,9.0,9,7974,1,0,0,0,0,0,0,...,1,0,0,7,2,77,18,85,7,0
4,10.0,9,7974,1,0,0,0,0,0,0,...,1,0,0,7,2,77,18,85,7,0


In [83]:
from xgboost.sklearn import XGBClassifier
xgb_region = XGBClassifier(
 learning_rate =0.1,
 n_estimators=1000,
 max_depth=5,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
 seed=27)
xgb_region.fit(train_region,regn)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=1000,
       n_jobs=1, nthread=4, objective='multi:softprob', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=27, silent=True,
       subsample=0.8)

In [85]:
test.shape

(29, 1)

In [115]:
test_region.columns

Index(['active_deals', 'activity_count', 'buyer_book', 'servicing_contract',
       'cmbs', 'consultant', 'correspondent', 'foreign', 'master_servicer',
       'lender_book', 'loan_sales_book', 'loan_servicing',
       'num_deals_as_client', 'num_deals_as_investor', 'number_of_properties',
       'number_of_related_deals', 'number_of_related_properties',
       'number_of_won_deals_as_client', 'investor_type_Account',
       'investor_type_Advisor/Equity Fund', 'investor_type_Agency',
       'investor_type_Bank / Thrift', 'investor_type_Conduit/CMBS',
       'investor_type_Corporate Investor/User',
       'investor_type_Credit/Specialty Finance Company',
       'investor_type_Endowment', 'investor_type_Life Insurance Company',
       'investor_type_Pension Fund',
       'investor_type_Private Investor/Developer',
       'investor_type_Property Lender', 'investor_type_Property Owner',
       'investor_type_REIT', 'investor_type_Sovereign Wealth Fund',
       'investor_type_Special Servi

In [116]:
test_region.head()

,active_deals,activity_count,buyer_book,servicing_contract,cmbs,consultant,correspondent,foreign,master_servicer,lender_book,...,investor_type_Life Insurance Company,investor_type_Pension Fund,investor_type_Private Investor/Developer,investor_type_Property Lender,investor_type_Property Owner,investor_type_REIT,investor_type_Sovereign Wealth Fund,investor_type_Special Servicer,investor_type_Tenants in Common,investor_type_Unknown
0,113,203365,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,261,127948,1,0,0,0,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0
2,379,204360,1,0,0,0,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0
3,284,132408,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,299,11365,1,1,0,0,1,0,0,1,...,1,0,0,0,0,0,0,0,0,0


In [105]:
train_region.columns

Index(['Unnamed: 0', 'active_deals', 'activity_count', 'buyer_book', 'cmbs',
       'consultant', 'correspondent', 'foreign', 'investor_type_Account',
       'investor_type_Advisor/Equity Fund', 'investor_type_Agency',
       'investor_type_Bank / Thrift', 'investor_type_Conduit/CMBS',
       'investor_type_Corporate Investor/User',
       'investor_type_Credit/Specialty Finance Company',
       'investor_type_Endowment', 'investor_type_Life Insurance Company',
       'investor_type_Pension Fund',
       'investor_type_Private Investor/Developer',
       'investor_type_Property Lender', 'investor_type_Property Owner',
       'investor_type_REIT', 'investor_type_Sovereign Wealth Fund',
       'investor_type_Special Servicer', 'investor_type_Tenants in Common',
       'investor_type_Unknown', 'lender_book', 'loan_sales_book',
       'loan_servicing', 'master_servicer', 'num_deals_as_client',
       'num_deals_as_investor', 'number_of_properties',
       'number_of_related_deals', 'number

In [97]:
test_region.to_csv('test_region.csv')

In [98]:
test_region=pd.read_csv('test_region.csv')

In [118]:
test_region.head()

,active_deals,activity_count,buyer_book,servicing_contract,cmbs,consultant,correspondent,foreign,master_servicer,lender_book,...,investor_type_Pension Fund,investor_type_Private Investor/Developer,investor_type_Property Lender,investor_type_Property Owner,investor_type_REIT,investor_type_Sovereign Wealth Fund,investor_type_Special Servicer,investor_type_Tenants in Common,investor_type_Unknown,Unnamed: 0
0,113,203365,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,261,127948,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,379,204360,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,284,132408,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,299,11365,1,1,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [117]:
test_region['Unnamed: 0']=0

In [101]:
#test_region.drop('Unnamed: 0.1', axis=1, inplace=True)

In [102]:
test_region.shape

(29, 37)

In [121]:
columnsTitles = ['Unnamed: 0', 'active_deals', 'activity_count', 'buyer_book', 'cmbs',
       'consultant', 'correspondent', 'foreign', 'investor_type_Account',
       'investor_type_Advisor/Equity Fund', 'investor_type_Agency',
       'investor_type_Bank / Thrift', 'investor_type_Conduit/CMBS',
       'investor_type_Corporate Investor/User',
       'investor_type_Credit/Specialty Finance Company',
       'investor_type_Endowment', 'investor_type_Life Insurance Company',
       'investor_type_Pension Fund',
       'investor_type_Private Investor/Developer',
       'investor_type_Property Lender', 'investor_type_Property Owner',
       'investor_type_REIT', 'investor_type_Sovereign Wealth Fund',
       'investor_type_Special Servicer', 'investor_type_Tenants in Common',
       'investor_type_Unknown', 'lender_book', 'loan_sales_book',
       'loan_servicing', 'master_servicer', 'num_deals_as_client',
       'num_deals_as_investor', 'number_of_properties',
       'number_of_related_deals', 'number_of_related_properties',
       'number_of_won_deals_as_client', 'servicing_contract']

test_region=test_region.reindex(columns=columnsTitles)

In [122]:
test_region.head()

,Unnamed: 0,active_deals,activity_count,buyer_book,cmbs,consultant,correspondent,foreign,investor_type_Account,investor_type_Advisor/Equity Fund,...,loan_sales_book,loan_servicing,master_servicer,num_deals_as_client,num_deals_as_investor,number_of_properties,number_of_related_deals,number_of_related_properties,number_of_won_deals_as_client,servicing_contract
0,0,113,203365,1,0,0,0,0,0,1,...,1,0,0,66,47,330,174,383,47,0
1,0,261,127948,1,0,0,0,0,0,0,...,1,0,0,40,221,138,283,276,30,0
2,0,379,204360,1,0,0,0,0,0,0,...,1,0,0,92,287,86,478,359,72,0
3,0,284,132408,1,0,0,0,0,0,1,...,1,0,0,224,60,231,384,260,181,0
4,0,299,11365,1,0,0,1,0,0,0,...,1,0,0,12,287,11,294,98,9,1


In [123]:
pred_test_region=xgb_region.predict(test_region)

C:\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [124]:
sub_region = pd.DataFrame({'id_accs':test['id_accs'],'region__c':pred_test_region})

In [133]:
sub_region.to_csv('testsubregion.csv')

In [127]:
sub_region['region__c'].unique()

array(['Southeast', 'Southwest', 'Midwest', 'Northeast'], dtype=object)

In [128]:
prop.head()

,id_props,building_status,building_tax_expenses,id_deals,city,class,county,sale_status,portfolio,market,...,price_per_sq_ft,price_per_unit,property_type_1,region__c,sale_amount__c,sale_date__c,size_acres,size_sf,size_units,year_built
0,a0I2A00000XQGKlUAP,Existing,2009 Tax @ $1.25/sf,a0I2A00000XQGKlUAP,City-0488,Class B,County-0172,N,False,Market-0059,...,46.59,18350000.00,Industrial,Southwest,1.835000e+07,07-10-2015,3.760,393862.0,1.0,1980.0
1,a0I2A00000XQGKnUAP,NaN,NaN,a0I2A00000XQGKnUAP,City-1975,Class A,County-0465,Y,True,Market-0239,...,1141.55,2109090.91,Multi-Housing,Southeast,2.144058e+07,12/31/2016,19.600,508080.0,275.0,2014.0
2,a0I2A00000XQGKoUAP,Existing,"2016 Tax @ $5.63/sf; 2007 Ops @ $19.58/sf, 201...",a0I2A00000XQGKoUAP,City-1415,Class B,County-0452,N,False,Market-0165,...,1506.65,NaN,Retail,Northeast,2.754156e+07,9/30/2016,0.266,18280.0,NaN,1902.0
3,a0I2A00000XQGKpUAP,NaN,NaN,a0I2A00000XQGKpUAP,City-1415,Class A,County-0452,N,False,Market-0165,...,580.83,1220689.66,NaN,Northeast,1.416000e+08,1/21/2016,0.530,243789.0,116.0,NaN
4,a0I2A00000XQGKqUAP,NaN,NaN,a0I2A00000XQGKqUAP,City-0941,NaN,County-0284,N,False,Market-0106,...,NaN,NaN,Self-Storage,Southwest,0.000000e+00,9/30/2014,NaN,NaN,NaN,NaN


In [129]:
submission1=sub_region.merge(prop[['region__c','id_props']], on=['region__c'], how='left')

In [131]:
submission1.to_csv('submission_region.csv')

In [203]:
oppo.head()

,id_deals,id_accs,accounting_date,best_initial_bid,id_deals.1,closedate,date_closed,deal_type,deal_update_flag,debt_yield,fiscal,fiscalquarter,fiscalyear,platform,property_group,property_type
0,0062A00000moF1LQAU,0012A000023XvIOQA0,NaN,NaN,0062A00000moF1LQAU,08-12-2005,8/18/2005 0:00,Fixed-rate,True,NaN,2005 3,3,2005,Debt,Healthcare,Healthcare
1,0062A00000moF9LQAU,0012A000023YFe1QAG,NaN,NaN,0062A00000moF9LQAU,02-06-2012,6/27/2012 12:00,Fixed-rate,True,NaN,2012 1,1,2012,Debt,Healthcare,Biotech
2,0062A00000moGodQAE,0012A000023YEDcQAO,08-12-2004,NaN,0062A00000moGodQAE,11/24/2003,08-11-2004 00:00,Construction Loan,True,NaN,2003 4,4,2003,Debt,Healthcare,Biotech
3,0062A00000moBTHQA2,0012A000023YA3nQAG,NaN,NaN,0062A00000moBTHQA2,02-10-2006,4/19/2006 0:00,Fixed-rate,True,NaN,2006 1,1,2006,Debt,Healthcare,Healthcare
4,0062A00000mo8OjQAI,0012A000023XpRUQA0,NaN,NaN,0062A00000mo8OjQAI,7/13/2006,11/16/2006 0:00,Fixed-rate,True,NaN,2006 3,3,2006,Debt,Healthcare,Healthcare


In [136]:
oppotemp = pd.DataFrame({'id_deals':oppo['id_deals'],'deal_update_flag':oppo['deal_update_flag']})

In [137]:
oppotemp.head()

,deal_update_flag,id_deals
0,True,0062A00000moF1LQAU
1,True,0062A00000moF9LQAU
2,True,0062A00000moGodQAE
3,True,0062A00000moBTHQA2
4,True,0062A00000mo8OjQAI


In [138]:
proptemp = pd.DataFrame({'id_deals':prop['id_deals']})

In [139]:
proptemp.head()

,id_deals
0,a0I2A00000XQGKlUAP
1,a0I2A00000XQGKnUAP
2,a0I2A00000XQGKoUAP
3,a0I2A00000XQGKpUAP
4,a0I2A00000XQGKqUAP


In [140]:
proptemp = proptemp.merge(oppotemp,on=['id_deals'],how='left')

In [141]:
proptemp.head()

,id_deals,deal_update_flag
0,a0I2A00000XQGKlUAP,NaN
1,a0I2A00000XQGKnUAP,NaN
2,a0I2A00000XQGKoUAP,NaN
3,a0I2A00000XQGKpUAP,NaN
4,a0I2A00000XQGKqUAP,NaN


In [145]:
proptemp.shape

(18298, 2)

In [142]:
l = proptemp['deal_update_flag'].isnull()

In [143]:
l.size

18298

In [148]:
dealtoproptemp = pd.DataFrame({'id_deals':dealtoprop['id_deals']})

In [149]:
dealtoproptemp = dealtoproptemp.merge(oppotemp,on=['id_deals'],how='left')

In [154]:
dealtoproptemp.shape

(25820, 2)

In [152]:
l = dealtoproptemp['deal_update_flag'].isnull()

In [153]:
l.size

25820

In [169]:
acc_prop.head()

,id,id_deals,id_accs,id_props,active_deals,activity_count,buyer_book,servicing_contract,cmbs,consultant,...,investor_type_Life Insurance Company,investor_type_Pension Fund,investor_type_Private Investor/Developer,investor_type_Property Lender,investor_type_Property Owner,investor_type_REIT,investor_type_Sovereign Wealth Fund,investor_type_Special Servicer,investor_type_Tenants in Common,investor_type_Unknown
0,a062A00001Whq5tQAB,a062A00001Whq5tQAB,0012A000023XlCCQA0,a0I2A00000XQKwXUAX,1,2367,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,a062A00001VmP7EQAV,a062A00001VmP7EQAV,0012A000023XmOxQAK,a0I2A00000XvUEwUAN,9,7974,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,a062A00001VmQByQAN,a062A00001VmQByQAN,0012A000023XmOxQAK,a0I2A00000XwQ13UAF,9,7974,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,a062A00001VmQC2QAN,a062A00001VmQC2QAN,0012A000023XmOxQAK,a0I2A00000XwZyBUAV,9,7974,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,a062A00001VmW8wQAF,a062A00001VmW8wQAF,0012A000023XmOxQAK,a0I2A00000ZGZClUAP,9,7974,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [175]:
acc_prop.to_csv('acc_prop.csv')

In [168]:
acc_prop = acc_prop.merge(acc,on=['id_accs'],how='left')

In [174]:
test.head()

,id_accs,active_deals,activity_count,buyer_book,servicing_contract,cmbs,consultant,correspondent,foreign,master_servicer,...,investor_type_Life Insurance Company,investor_type_Pension Fund,investor_type_Private Investor/Developer,investor_type_Property Lender,investor_type_Property Owner,investor_type_REIT,investor_type_Sovereign Wealth Fund,investor_type_Special Servicer,investor_type_Tenants in Common,investor_type_Unknown
0,0012A00002AXHJUQA5,113,203365,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0012A000023XlWaQAK,261,127948,1,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,0012A000023Y4bYQAS,379,204360,1,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3,0012A000023XlbuQAC,284,132408,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0012A000023Xlh9QAC,299,11365,1,1,0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,0


In [173]:
test = test.merge(acc,on=['id_accs'],how='left')

In [177]:
acc_prop.columns

Index(['id', 'id_deals', 'id_accs', 'id_props', 'active_deals',
       'activity_count', 'buyer_book', 'servicing_contract', 'cmbs',
       'consultant', 'correspondent', 'foreign', 'master_servicer',
       'lender_book', 'loan_sales_book', 'loan_servicing',
       'num_deals_as_client', 'num_deals_as_investor', 'number_of_properties',
       'number_of_related_deals', 'number_of_related_properties',
       'number_of_won_deals_as_client', 'investor_type_Account',
       'investor_type_Advisor/Equity Fund', 'investor_type_Agency',
       'investor_type_Bank / Thrift', 'investor_type_Conduit/CMBS',
       'investor_type_Corporate Investor/User',
       'investor_type_Credit/Specialty Finance Company',
       'investor_type_Endowment', 'investor_type_Life Insurance Company',
       'investor_type_Pension Fund',
       'investor_type_Private Investor/Developer',
       'investor_type_Property Lender', 'investor_type_Property Owner',
       'investor_type_REIT', 'investor_type_Sovereign W

In [179]:
test = test.merge(acc_prop[['id_props','buyer_book' , 'servicing_contract' , 'cmbs' , 'consultant' ,'correspondent' , 'foreign' , 'master_servicer' , 'lender_book'  , 'loan_sales_book' , 'loan_servicing','investor_type_Account',
       'investor_type_Advisor/Equity Fund', 'investor_type_Agency',
       'investor_type_Bank / Thrift', 'investor_type_Conduit/CMBS',
       'investor_type_Corporate Investor/User',
       'investor_type_Credit/Specialty Finance Company',
       'investor_type_Endowment', 'investor_type_Life Insurance Company',
       'investor_type_Pension Fund',
       'investor_type_Private Investor/Developer',
       'investor_type_Property Lender', 'investor_type_Property Owner',
       'investor_type_REIT', 'investor_type_Sovereign Wealth Fund',
       'investor_type_Special Servicer', 'investor_type_Tenants in Common',
       'investor_type_Unknown']] , on=['buyer_book' , 'servicing_contract' , 'cmbs' , 'consultant' ,'correspondent' , 'foreign' , 'master_servicer' , 'lender_book'  , 'loan_sales_book' , 'loan_servicing','investor_type_Account',
       'investor_type_Advisor/Equity Fund', 'investor_type_Agency',
       'investor_type_Bank / Thrift', 'investor_type_Conduit/CMBS',
       'investor_type_Corporate Investor/User',
       'investor_type_Credit/Specialty Finance Company',
       'investor_type_Endowment', 'investor_type_Life Insurance Company',
       'investor_type_Pension Fund',
       'investor_type_Private Investor/Developer',
       'investor_type_Property Lender', 'investor_type_Property Owner',
       'investor_type_REIT', 'investor_type_Sovereign Wealth Fund',
       'investor_type_Special Servicer', 'investor_type_Tenants in Common',
       'investor_type_Unknown'], how = 'left')

In [205]:
test.shape

(72621, 38)

In [183]:
testsub = pd.DataFrame({'id_accs':test['id_accs'],'id_prop':test['id_props']})

In [184]:
testsub.head()

,id_accs,id_prop
0,0012A00002AXHJUQA5,a0I2A00000Y65dkUAB
1,0012A00002AXHJUQA5,a0I2A00000XwWtdUAF
2,0012A00002AXHJUQA5,a0I2A00000XQHddUAH
3,0012A00002AXHJUQA5,a0I2A00000XQIUYUA5
4,0012A00002AXHJUQA5,a0I2A00000XQIiRUAX


In [ ]:
testsub = pd.concat([])

In [185]:
testsub.to_csv('testsub.csv')

In [188]:
test_s_p_r.head()

,id_accs,sale_status,portfolio,region__c
0,0012A00002AXHJUQA5,N,False,Southeast
1,0012A000023XlWaQAK,N,False,Southeast
2,0012A000023Y4bYQAS,N,False,Southwest
3,0012A000023XlbuQAC,N,False,Southeast
4,0012A000023Xlh9QAC,N,False,Southwest


In [189]:
prop.head()

,id_props,building_status,building_tax_expenses,id_deals,city,class,county,sale_status,portfolio,market,...,price_per_sq_ft,price_per_unit,property_type_1,region__c,sale_amount__c,sale_date__c,size_acres,size_sf,size_units,year_built
0,a0I2A00000XQGKlUAP,Existing,2009 Tax @ $1.25/sf,a0I2A00000XQGKlUAP,City-0488,Class B,County-0172,N,False,Market-0059,...,46.59,18350000.00,Industrial,Southwest,1.835000e+07,07-10-2015,3.760,393862.0,1.0,1980.0
1,a0I2A00000XQGKnUAP,NaN,NaN,a0I2A00000XQGKnUAP,City-1975,Class A,County-0465,Y,True,Market-0239,...,1141.55,2109090.91,Multi-Housing,Southeast,2.144058e+07,12/31/2016,19.600,508080.0,275.0,2014.0
2,a0I2A00000XQGKoUAP,Existing,"2016 Tax @ $5.63/sf; 2007 Ops @ $19.58/sf, 201...",a0I2A00000XQGKoUAP,City-1415,Class B,County-0452,N,False,Market-0165,...,1506.65,NaN,Retail,Northeast,2.754156e+07,9/30/2016,0.266,18280.0,NaN,1902.0
3,a0I2A00000XQGKpUAP,NaN,NaN,a0I2A00000XQGKpUAP,City-1415,Class A,County-0452,N,False,Market-0165,...,580.83,1220689.66,NaN,Northeast,1.416000e+08,1/21/2016,0.530,243789.0,116.0,NaN
4,a0I2A00000XQGKqUAP,NaN,NaN,a0I2A00000XQGKqUAP,City-0941,NaN,County-0284,N,False,Market-0106,...,NaN,NaN,Self-Storage,Southwest,0.000000e+00,9/30/2014,NaN,NaN,NaN,NaN


In [190]:
test3prop = test_s_p_r.merge(prop[['id_props','sale_status','portfolio','region__c']], on = ['sale_status','portfolio','region__c'] , how='left')

In [196]:
test3prop.head()

,id_accs,id_prop
0,0012A00002AXHJUQA5,a0I2A00000XQGKxUAP
1,0012A00002AXHJUQA5,a0I2A00000XQGKzUAP
2,0012A00002AXHJUQA5,a0I2A00000XQGLCUA5
3,0012A00002AXHJUQA5,a0I2A00000XQGLWUA5
4,0012A00002AXHJUQA5,a0I2A00000XQGLiUAP


In [193]:
test3prop['id_prop'] = test3prop['id_props']

In [195]:
test3prop.drop(['sale_status','portfolio','region__c','id_props'],axis=1,inplace=True)

In [200]:
test3prop.shape

(52213, 2)

In [197]:
testsub = pd.concat([testsub,test3prop],axis=0).drop_duplicates().reset_index(drop=True)

In [198]:
testsub.shape

(120494, 2)

In [206]:
oppo.head()

,id_deals,id_accs,accounting_date,best_initial_bid,id_deals.1,closedate,date_closed,deal_type,deal_update_flag,debt_yield,fiscal,fiscalquarter,fiscalyear,platform,property_group,property_type
0,0062A00000moF1LQAU,0012A000023XvIOQA0,NaN,NaN,0062A00000moF1LQAU,08-12-2005,8/18/2005 0:00,Fixed-rate,True,NaN,2005 3,3,2005,Debt,Healthcare,Healthcare
1,0062A00000moF9LQAU,0012A000023YFe1QAG,NaN,NaN,0062A00000moF9LQAU,02-06-2012,6/27/2012 12:00,Fixed-rate,True,NaN,2012 1,1,2012,Debt,Healthcare,Biotech
2,0062A00000moGodQAE,0012A000023YEDcQAO,08-12-2004,NaN,0062A00000moGodQAE,11/24/2003,08-11-2004 00:00,Construction Loan,True,NaN,2003 4,4,2003,Debt,Healthcare,Biotech
3,0062A00000moBTHQA2,0012A000023YA3nQAG,NaN,NaN,0062A00000moBTHQA2,02-10-2006,4/19/2006 0:00,Fixed-rate,True,NaN,2006 1,1,2006,Debt,Healthcare,Healthcare
4,0062A00000mo8OjQAI,0012A000023XpRUQA0,NaN,NaN,0062A00000mo8OjQAI,7/13/2006,11/16/2006 0:00,Fixed-rate,True,NaN,2006 3,3,2006,Debt,Healthcare,Healthcare


In [237]:
acc_prop_deals1=acc_prop.merge(dealtoprop[['id_deals','id_props']], on=['id_props'], how='left')

In [232]:
dealtoprop.head()

,id,createdbyid,id_deals,id_props
0,a0A2A00000TxIrIUAV,005F0000007WnNDIA0,0062A00000o979GQAQ,a0I2A00000XQH8YUAX
1,a0A2A00000TxIrSUAV,005F0000007WnNDIA0,0062A00000o94JrQAI,a0I2A00000XQJ4dUAH
2,a0A2A00000TxIrXUAV,005F0000007WnNDIA0,0062A00000o94GJQAY,a0I2A00000XQJL0UAP
3,a0A2A00000TxIrcUAF,005F0000007WnNDIA0,0062A00000o982fQAA,a0I2A00000XQKgkUAH
4,a0A2A00000TxIrhUAF,005F0000007WnNDIA0,0062A00000o96M4QAI,a0I2A00000XQMEhUAP


In [249]:
acc_prop_deals1.head()

,id,id_deals_x,id_accs,id_props,active_deals,activity_count,buyer_book,servicing_contract,cmbs,consultant,...,investor_type_Pension Fund,investor_type_Private Investor/Developer,investor_type_Property Lender,investor_type_Property Owner,investor_type_REIT,investor_type_Sovereign Wealth Fund,investor_type_Special Servicer,investor_type_Tenants in Common,investor_type_Unknown,id_deals_y
0,a062A00001Whq5tQAB,a062A00001Whq5tQAB,0012A000023XlCCQA0,a0I2A00000XQKwXUAX,1,2367,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0062A00000moAuRQAU
1,a062A00001VmP7EQAV,a062A00001VmP7EQAV,0012A000023XmOxQAK,a0I2A00000XvUEwUAN,9,7974,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0062A00000okwMRQAY
2,a062A00001VmP7EQAV,a062A00001VmP7EQAV,0012A000023XmOxQAK,a0I2A00000XvUEwUAN,9,7974,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0062A00000pd1e0QAA
3,a062A00001VmP7EQAV,a062A00001VmP7EQAV,0012A000023XmOxQAK,a0I2A00000XvUEwUAN,9,7974,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0062A00000moPGGQA2
4,a062A00001VmQByQAN,a062A00001VmQByQAN,0012A000023XmOxQAK,a0I2A00000XwQ13UAF,9,7974,1,0,0,0,...,0,0,0,0,1,0,0,0,0,NaN


In [250]:
acc_prop_deals1.shape

(80056, 41)

In [247]:
l2=acc_prop_deals1[acc_prop_deals1['id_deals_y'].isnull()]

In [253]:
l3=acc_prop_deals1[acc_prop_deals1['id_deals_y'].notnull()]

In [261]:
l3.head()

,id,id_accs,id_props,active_deals,activity_count,buyer_book,servicing_contract,cmbs,consultant,correspondent,...,investor_type_Pension Fund,investor_type_Private Investor/Developer,investor_type_Property Lender,investor_type_Property Owner,investor_type_REIT,investor_type_Sovereign Wealth Fund,investor_type_Special Servicer,investor_type_Tenants in Common,investor_type_Unknown,id_deals
0,a062A00001Whq5tQAB,0012A000023XlCCQA0,a0I2A00000XQKwXUAX,1,2367,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0062A00000moAuRQAU
1,a062A00001VmP7EQAV,0012A000023XmOxQAK,a0I2A00000XvUEwUAN,9,7974,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0062A00000okwMRQAY
2,a062A00001VmP7EQAV,0012A000023XmOxQAK,a0I2A00000XvUEwUAN,9,7974,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0062A00000pd1e0QAA
3,a062A00001VmP7EQAV,0012A000023XmOxQAK,a0I2A00000XvUEwUAN,9,7974,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0062A00000moPGGQA2
10,a062A00001WhlUYQAZ,0012A000023XmOxQAK,a0I2A00000XQHk4UAH,9,7974,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0062A00000moET5QAM


In [252]:
l2.shape

(48257, 41)

In [254]:
l3.shape

(31799, 41)

In [241]:
acc_prop_deals1.to_csv('acc_prop_deals1.csv')

In [258]:
l3_orig=l3.copy()

In [262]:
l3=l3.merge(oppo[['id_accs','id_deals','property_group']], on=['id_accs','id_deals'], how='left' )

In [263]:
l3.head()

,id,id_accs,id_props,active_deals,activity_count,buyer_book,servicing_contract,cmbs,consultant,correspondent,...,investor_type_Private Investor/Developer,investor_type_Property Lender,investor_type_Property Owner,investor_type_REIT,investor_type_Sovereign Wealth Fund,investor_type_Special Servicer,investor_type_Tenants in Common,investor_type_Unknown,id_deals,property_group
0,a062A00001Whq5tQAB,0012A000023XlCCQA0,a0I2A00000XQKwXUAX,1,2367,1,0,0,0,0,...,1,0,0,0,0,0,0,0,0062A00000moAuRQAU,NaN
1,a062A00001VmP7EQAV,0012A000023XmOxQAK,a0I2A00000XvUEwUAN,9,7974,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0062A00000okwMRQAY,NaN
2,a062A00001VmP7EQAV,0012A000023XmOxQAK,a0I2A00000XvUEwUAN,9,7974,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0062A00000pd1e0QAA,NaN
3,a062A00001VmP7EQAV,0012A000023XmOxQAK,a0I2A00000XvUEwUAN,9,7974,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0062A00000moPGGQA2,Hotel-Lodging
4,a062A00001WhlUYQAZ,0012A000023XmOxQAK,a0I2A00000XQHk4UAH,9,7974,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0062A00000moET5QAM,NaN


In [265]:
l3.shape

(31799, 41)

In [266]:
l3['property_group'].describe()

count              5357
unique               10
top       Multi-Housing
freq               1950
Name: property_group, dtype: object

In [267]:
prop.head()

,id_props,building_status,building_tax_expenses,id_deals,city,class,county,sale_status,portfolio,market,...,price_per_sq_ft,price_per_unit,property_type_1,region__c,sale_amount__c,sale_date__c,size_acres,size_sf,size_units,year_built
0,a0I2A00000XQGKlUAP,Existing,2009 Tax @ $1.25/sf,a0I2A00000XQGKlUAP,City-0488,Class B,County-0172,N,False,Market-0059,...,46.59,18350000.00,Industrial,Southwest,1.835000e+07,07-10-2015,3.760,393862.0,1.0,1980.0
1,a0I2A00000XQGKnUAP,NaN,NaN,a0I2A00000XQGKnUAP,City-1975,Class A,County-0465,Y,True,Market-0239,...,1141.55,2109090.91,Multi-Housing,Southeast,2.144058e+07,12/31/2016,19.600,508080.0,275.0,2014.0
2,a0I2A00000XQGKoUAP,Existing,"2016 Tax @ $5.63/sf; 2007 Ops @ $19.58/sf, 201...",a0I2A00000XQGKoUAP,City-1415,Class B,County-0452,N,False,Market-0165,...,1506.65,NaN,Retail,Northeast,2.754156e+07,9/30/2016,0.266,18280.0,NaN,1902.0
3,a0I2A00000XQGKpUAP,NaN,NaN,a0I2A00000XQGKpUAP,City-1415,Class A,County-0452,N,False,Market-0165,...,580.83,1220689.66,NaN,Northeast,1.416000e+08,1/21/2016,0.530,243789.0,116.0,NaN
4,a0I2A00000XQGKqUAP,NaN,NaN,a0I2A00000XQGKqUAP,City-0941,NaN,County-0284,N,False,Market-0106,...,NaN,NaN,Self-Storage,Southwest,0.000000e+00,9/30/2014,NaN,NaN,NaN,NaN


In [270]:
leng=prop['year_built'].unique()
leng.size

168

In [271]:
acc_prop.head()

,id,id_deals,id_accs,id_props,active_deals,activity_count,buyer_book,servicing_contract,cmbs,consultant,...,investor_type_Life Insurance Company,investor_type_Pension Fund,investor_type_Private Investor/Developer,investor_type_Property Lender,investor_type_Property Owner,investor_type_REIT,investor_type_Sovereign Wealth Fund,investor_type_Special Servicer,investor_type_Tenants in Common,investor_type_Unknown
0,a062A00001Whq5tQAB,a062A00001Whq5tQAB,0012A000023XlCCQA0,a0I2A00000XQKwXUAX,1,2367,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,a062A00001VmP7EQAV,a062A00001VmP7EQAV,0012A000023XmOxQAK,a0I2A00000XvUEwUAN,9,7974,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,a062A00001VmQByQAN,a062A00001VmQByQAN,0012A000023XmOxQAK,a0I2A00000XwQ13UAF,9,7974,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,a062A00001VmQC2QAN,a062A00001VmQC2QAN,0012A000023XmOxQAK,a0I2A00000XwZyBUAV,9,7974,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,a062A00001VmW8wQAF,a062A00001VmW8wQAF,0012A000023XmOxQAK,a0I2A00000ZGZClUAP,9,7974,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [272]:
acc_prop.to_csv('acc_prop.csv')

In [273]:
y_train = acc_prop['id_props']

In [275]:
y_train.to_csv('y_train.csv')

In [278]:
#l = pd.DataFrame({'id_props':y_train.unique()})
l.to_csv('l.csv')

In [215]:
acc_prop_oppo2=pd.DataFrame()

In [216]:
acc_prop_oppo2['id_accs']=oppo['id_accs']
acc_prop_oppo2['deal_update_flag']=oppo['deal_update_flag']

In [217]:
acc_prop_oppo2.head()

,id_accs,deal_update_flag
0,0012A000023XvIOQA0,True
1,0012A000023YFe1QAG,True
2,0012A000023YEDcQAO,True
3,0012A000023YA3nQAG,True
4,0012A000023XpRUQA0,True


In [219]:
acc_prop_oppo3=acc_prop.merge(acc_prop_oppo2, on=['id_accs'], how='left')

In [220]:
acc_prop_oppo3.shape

(954232, 41)

In [208]:
acc_prop_oppo.head()

,id,id_deals,id_accs,id_props,active_deals,activity_count,buyer_book,servicing_contract,cmbs,consultant,...,investor_type_Pension Fund,investor_type_Private Investor/Developer,investor_type_Property Lender,investor_type_Property Owner,investor_type_REIT,investor_type_Sovereign Wealth Fund,investor_type_Special Servicer,investor_type_Tenants in Common,investor_type_Unknown,deal_update_flag
0,a062A00001Whq5tQAB,a062A00001Whq5tQAB,0012A000023XlCCQA0,a0I2A00000XQKwXUAX,1,2367,1,0,0,0,...,0,1,0,0,0,0,0,0,0,NaN
1,a062A00001VmP7EQAV,a062A00001VmP7EQAV,0012A000023XmOxQAK,a0I2A00000XvUEwUAN,9,7974,1,0,0,0,...,0,0,0,0,1,0,0,0,0,True
2,a062A00001VmP7EQAV,a062A00001VmP7EQAV,0012A000023XmOxQAK,a0I2A00000XvUEwUAN,9,7974,1,0,0,0,...,0,0,0,0,1,0,0,0,0,True
3,a062A00001VmP7EQAV,a062A00001VmP7EQAV,0012A000023XmOxQAK,a0I2A00000XvUEwUAN,9,7974,1,0,0,0,...,0,0,0,0,1,0,0,0,0,True
4,a062A00001VmP7EQAV,a062A00001VmP7EQAV,0012A000023XmOxQAK,a0I2A00000XvUEwUAN,9,7974,1,0,0,0,...,0,0,0,0,1,0,0,0,0,True


In [223]:
l=acc_prop_oppo['deal_update_flag'].isnull()
l.size

954232

In [225]:
acc_prop_oppo['deal_update_flag'].unique()

array([nan, True, False], dtype=object)

In [226]:
acc_prop_oppo['deal_update_flag'].describe()

count     932192
unique         2
top         True
freq      929790
Name: deal_update_flag, dtype: object

In [12]:
oppo.head(3)

,id_deals,id_accs,id_deals.1,deal_type,deal_update_flag,debt_yield,platform,property_group,property_type
0,0062A00000moF1LQAU,0012A000023XvIOQA0,0062A00000moF1LQAU,Fixed-rate,True,NaN,Debt,Healthcare,Healthcare
1,0062A00000moF9LQAU,0012A000023YFe1QAG,0062A00000moF9LQAU,Fixed-rate,True,NaN,Debt,Healthcare,Biotech
2,0062A00000moGodQAE,0012A000023YEDcQAO,0062A00000moGodQAE,Construction Loan,True,NaN,Debt,Healthcare,Biotech


In [7]:
oppo['property_type'].unique()

array(['Healthcare', 'Biotech', 'Seniors Housing', 'Hotel/Lodging',
       'Industrial', 'Land', 'Mixed Use', 'Apartments',
       'Manufactured Home Communities', 'SFR - Attached',
       'SFR - Detached', 'Office', 'Data Center', 'Unassigned',
       'Parking Garage', 'Recreational/Speciality', 'Promissory Note',
       'Day Care', 'Retail', 'Self Storage', nan], dtype=object)

In [8]:
oppo['platform'].unique()

array(['Debt', 'Equity Placement', 'Investment Sales', 'Loan Sales',
       'Securities', nan], dtype=object)

In [9]:
##############We have removed the non categorical values from the dataframe
oppo.drop(['id_deals.1','accounting_date','best_initial_bid','closedate','date_closed','fiscal','fiscalyear','fiscalquarter'], axis=1, inplace=True)

In [39]:
oppo.shape

(26029, 9)

In [64]:
###############We merge the two dataframes acc and oppo to include the features of the oppo
acc_oppo = acc.merge(oppo,on=['id_accs'], how='left')

In [65]:
acc_oppo.head(3)

,id_accs,active_deals,activity_count,buyer_book,servicing_contract,cmbs,consultant,correspondent,foreign,master_servicer,...,investor_type_Tenants in Common,investor_type_Unknown,id_deals,id_deals.1,deal_type,deal_update_flag,debt_yield,platform,property_group,property_type
0,0012A000023XlCCQA0,1,2367,1,0,0,0,0,0,0,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0012A000023XmOxQAK,9,7974,1,0,0,0,0,0,0,...,0,0,0062A00000moOejQAE,0062A00000moOejQAE,Floating-Rate,True,8.7,Debt,Hotel-Lodging,Hotel/Lodging
2,0012A000023XmOxQAK,9,7974,1,0,0,0,0,0,0,...,0,0,0062A00000nBEazQAG,0062A00000nBEazQAG,Floating-Rate,True,0.0,Debt,Hotel-Lodging,Hotel/Lodging


In [50]:
acc_oppo.describe()

,active_deals,activity_count,buyer_book,servicing_contract,cmbs,consultant,correspondent,foreign,master_servicer,lender_book,...,investor_type_Pension Fund,investor_type_Private Investor/Developer,investor_type_Property Lender,investor_type_Property Owner,investor_type_REIT,investor_type_Sovereign Wealth Fund,investor_type_Special Servicer,investor_type_Tenants in Common,investor_type_Unknown,debt_yield
count,12731.000000,12731.000000,12731.000000,12731.000000,12731.000000,12731.0,12731.000000,12731.000000,12731.000000,12731.000000,...,12731.000000,12731.000000,12731.000000,12731.000000,12731.000000,12731.000000,12731.000000,12731.000000,12731.000000,7353.000000
mean,75.648025,29791.675516,0.939989,0.053020,0.001728,0.0,0.006755,0.112717,0.000628,0.546933,...,0.004085,0.533972,0.000079,0.010447,0.103213,0.000157,0.004163,0.000079,0.008405,0.907970
std,143.029544,38394.983966,0.237517,0.224083,0.041536,0.0,0.081915,0.316259,0.025061,0.497812,...,0.063782,0.498864,0.008863,0.101679,0.304248,0.012533,0.064390,0.008863,0.091295,2.823455
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,11.000000,5102.000000,1.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,33.000000,16677.000000,1.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,1.000000,...,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,80.000000,38971.000000,1.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,1.000000,...,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,2215.000000,217301.000000,1.000000,1.000000,1.000000,0.0,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,17.300000


In [66]:
acc_oppo['deal_type'].unique()

array([nan, 'Floating-Rate', 'Fixed-rate', 'Miscellaneous',
       'Property Sale', 'Well-Performing', 'Non-Performing',
       'Higher Risk Performing', 'Sub-Performing', 'Consulting Fees-Debt',
       'Joint Venture Equity', 'Subscription Facility', 'Advisory',
       'Mezzanine', 'Construction Loan', 'Discretionary Funds',
       'Consulting', 'Preferred Equity', 'B-Note', 'Accruing Loan',
       'Foreclosed/REO', 'Sale Leaseback', 'GP Equity',
       'Participating/Convertible', 'Bond Enhancement',
       'Entity Level-Equity'], dtype=object)

In [24]:
acc_oppo['platform'].unique()

array([nan, 'Debt', 'Investment Sales', 'Loan Sales', 'Equity Placement',
       'Securities'], dtype=object)

In [67]:
acc_oppo['deal_type'].describe()

count          11153
unique            25
top       Fixed-rate
freq            4795
Name: deal_type, dtype: object

In [27]:
acc_oppo['platform'].describe()

count     11153
unique        5
top        Debt
freq       7360
Name: platform, dtype: object

In [68]:
acc_oppo['deal_type'].fillna('Fixed-rate', inplace=True)

In [69]:
acc_oppo['platform'].fillna('Dept', inplace=True)

In [70]:
acc_oppo['property_group'].unique()

array([nan, 'Hotel-Lodging', 'Industrial', 'Land', 'Mixed Use',
       'Multi-Housing', 'Office', 'Other', 'Retail', 'Healthcare',
       'Self Storage'], dtype=object)

In [71]:
acc_oppo['property_group'].describe()

count             11152
unique               10
top       Multi-Housing
freq               3907
Name: property_group, dtype: object

In [72]:
acc_oppo['property_group'].fillna('Multi-Housing', inplace=True)

In [73]:
acc_oppo['property_type'].unique()

array([nan, 'Hotel/Lodging', 'Industrial', 'Land', 'Mixed Use',
       'Apartments', 'Manufactured Home Communities', 'SFR - Attached',
       'Office', 'Unassigned', 'Retail', 'Recreational/Speciality',
       'Healthcare', 'Self Storage', 'Seniors Housing', 'Biotech',
       'Promissory Note', 'Data Center', 'Parking Garage', 'Day Care'],
      dtype=object)

In [74]:
acc_oppo['property_type'].describe()

count          11152
unique            19
top       Apartments
freq            3789
Name: property_type, dtype: object

In [75]:
acc_oppo['property_type'].fillna('Apartments', inplace=True)

In [76]:
acc_oppo['deal_update_flag'].describe()

count     11154
unique        2
top        True
freq      11117
Name: deal_update_flag, dtype: object

In [77]:
acc_oppo['deal_update_flag'].fillna('True', inplace=True)

In [78]:
acc_oppo.drop('debt_yield', axis=1, inplace=True)

In [95]:
acc_oppo.head(3)

,id_accs,active_deals,activity_count,buyer_book,servicing_contract,cmbs,consultant,correspondent,foreign,master_servicer,...,investor_type_Special Servicer,investor_type_Tenants in Common,investor_type_Unknown,id_deals,id_deals.1,deal_type,deal_update_flag,platform,property_group,property_type
0,0012A000023XlCCQA0,1,2367,1,0,0,0,0,0,0,...,0,0,0,NaN,NaN,Fixed-rate,True,Dept,Multi-Housing,Apartments
1,0012A000023XmOxQAK,9,7974,1,0,0,0,0,0,0,...,0,0,0,0062A00000moOejQAE,0062A00000moOejQAE,Floating-Rate,True,Debt,Hotel-Lodging,Hotel/Lodging
2,0012A000023XmOxQAK,9,7974,1,0,0,0,0,0,0,...,0,0,0,0062A00000nBEazQAG,0062A00000nBEazQAG,Floating-Rate,True,Debt,Hotel-Lodging,Hotel/Lodging


In [96]:
acc_oppo.shape

(12731, 44)

,id_props,building_status,building_tax_expenses,id_deals,city,class,county,sale_status,portfolio,market,...,price_per_sq_ft,price_per_unit,property_type_1,region__c,sale_amount__c,sale_date__c,size_acres,size_sf,size_units,year_built
0,a0I2A00000XQGKlUAP,Existing,2009 Tax @ $1.25/sf,a0I2A00000XQGKlUAP,City-0488,Class B,County-0172,N,False,Market-0059,...,46.59,18350000.00,Industrial,Southwest,18350000.00,07-10-2015,3.760,393862.0,1.0,1980.0
1,a0I2A00000XQGKnUAP,NaN,NaN,a0I2A00000XQGKnUAP,City-1975,Class A,County-0465,Y,True,Market-0239,...,1141.55,2109090.91,Multi-Housing,Southeast,21440582.78,12/31/2016,19.600,508080.0,275.0,2014.0
2,a0I2A00000XQGKoUAP,Existing,"2016 Tax @ $5.63/sf; 2007 Ops @ $19.58/sf, 201...",a0I2A00000XQGKoUAP,City-1415,Class B,County-0452,N,False,Market-0165,...,1506.65,NaN,Retail,Northeast,27541555.35,9/30/2016,0.266,18280.0,NaN,1902.0


In [98]:
acc_oppo.columns

Index(['id_accs', 'active_deals', 'activity_count', 'buyer_book',
       'servicing_contract', 'cmbs', 'consultant', 'correspondent', 'foreign',
       'master_servicer', 'lender_book', 'loan_sales_book', 'loan_servicing',
       'num_deals_as_client', 'num_deals_as_investor', 'number_of_properties',
       'number_of_related_deals', 'number_of_related_properties',
       'number_of_won_deals_as_client', 'investor_type_Account',
       'investor_type_Advisor/Equity Fund', 'investor_type_Agency',
       'investor_type_Bank / Thrift', 'investor_type_Conduit/CMBS',
       'investor_type_Corporate Investor/User',
       'investor_type_Credit/Specialty Finance Company',
       'investor_type_Endowment', 'investor_type_Life Insurance Company',
       'investor_type_Pension Fund',
       'investor_type_Private Investor/Developer',
       'investor_type_Property Lender', 'investor_type_Property Owner',
       'investor_type_REIT', 'investor_type_Sovereign Wealth Fund',
       'investor_type_Sp

In [191]:
ap_acc_prop.head()

,id,id_deals,id_accs,id_props,active_deals,activity_count,buyer_book,servicing_contract,cmbs,consultant,...,investor_type_Pension Fund,investor_type_Private Investor/Developer,investor_type_Property Lender,investor_type_Property Owner,investor_type_REIT,investor_type_Sovereign Wealth Fund,investor_type_Special Servicer,investor_type_Tenants in Common,investor_type_Unknown,portfolio
0,a062A00001Whq5tQAB,a062A00001Whq5tQAB,0012A000023XlCCQA0,a0I2A00000XQKwXUAX,1,2367,1,0,0,0,...,0,1,0,0,0,0,0,0,0,False
1,a062A00001VmP7EQAV,a062A00001VmP7EQAV,0012A000023XmOxQAK,a0I2A00000XvUEwUAN,9,7974,1,0,0,0,...,0,0,0,0,1,0,0,0,0,False
2,a062A00001VmQByQAN,a062A00001VmQByQAN,0012A000023XmOxQAK,a0I2A00000XwQ13UAF,9,7974,1,0,0,0,...,0,0,0,0,1,0,0,0,0,NaN
3,a062A00001VmQC2QAN,a062A00001VmQC2QAN,0012A000023XmOxQAK,a0I2A00000XwZyBUAV,9,7974,1,0,0,0,...,0,0,0,0,1,0,0,0,0,NaN
4,a062A00001VmW8wQAF,a062A00001VmW8wQAF,0012A000023XmOxQAK,a0I2A00000ZGZClUAP,9,7974,1,0,0,0,...,0,0,0,0,1,0,0,0,0,NaN


In [192]:
ap_acc_prop['buyer_book'].unique()

array([1, 0], dtype=int64)

In [193]:
deals_prop_orig=deals_prop.copy()

In [199]:
prop['region__c'].unique()

array(['Southwest', 'Southeast', 'Northeast', 'West', 'Midwest',
       'Mid-Atlantic', nan], dtype=object)

In [194]:
deals_prop=deals_prop.merge(ap_acc_prop[['id_props']], on=['id_props'], how='left')

In [221]:
ap_acc_prop.describe()

,active_deals,activity_count,buyer_book,servicing_contract,cmbs,consultant,correspondent,foreign,master_servicer,lender_book,...,investor_type_Life Insurance Company,investor_type_Pension Fund,investor_type_Private Investor/Developer,investor_type_Property Lender,investor_type_Property Owner,investor_type_REIT,investor_type_Sovereign Wealth Fund,investor_type_Special Servicer,investor_type_Tenants in Common,investor_type_Unknown
count,70350.000000,70350.000000,70350.000000,70350.000000,70350.000000,70350.0,70350.000000,70350.000000,70350.000000,70350.000000,...,70350.000000,70350.000000,70350.000000,70350.000000,70350.000000,70350.000000,70350.000000,70350.000000,70350.000000,70350.000000
mean,115.627377,26353.855991,0.937655,0.113220,0.005330,0.0,0.036489,0.068273,0.000896,0.640824,...,0.040043,0.003241,0.350348,0.000085,0.037285,0.188330,0.000753,0.001393,0.000014,0.016347
std,319.144080,34556.066356,0.241784,0.316863,0.072816,0.0,0.187505,0.252215,0.029912,0.479762,...,0.196061,0.056837,0.477082,0.009235,0.189461,0.390978,0.027438,0.037298,0.003770,0.126806
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4.000000,3818.000000,1.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,14.000000,14953.000000,1.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,55.000000,30883.000000,1.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,1.000000,...,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,2215.000000,193100.000000,1.000000,1.000000,1.000000,0.0,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [224]:
ap_acc_prop.columns

Index(['id', 'id_deals', 'id_accs', 'id_props', 'active_deals',
       'activity_count', 'buyer_book', 'servicing_contract', 'cmbs',
       'consultant', 'correspondent', 'foreign', 'master_servicer',
       'lender_book', 'loan_sales_book', 'loan_servicing',
       'num_deals_as_client', 'num_deals_as_investor', 'number_of_properties',
       'number_of_related_deals', 'number_of_related_properties',
       'number_of_won_deals_as_client', 'investor_type_Account',
       'investor_type_Advisor/Equity Fund', 'investor_type_Agency',
       'investor_type_Bank / Thrift', 'investor_type_Conduit/CMBS',
       'investor_type_Corporate Investor/User',
       'investor_type_Credit/Specialty Finance Company',
       'investor_type_Endowment', 'investor_type_Life Insurance Company',
       'investor_type_Pension Fund',
       'investor_type_Private Investor/Developer',
       'investor_type_Property Lender', 'investor_type_Property Owner',
       'investor_type_REIT', 'investor_type_Sovereign W

In [248]:
ap_acc_prop['investor_type_Unknown'].describe()

count    70350.000000
mean         0.016347
std          0.126806
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          1.000000
Name: investor_type_Unknown, dtype: float64

In [ ]:
################predicting investor type

In [362]:
acc.head()

,id_accs,active_deals,activity_count,buyer_book,servicing_contract,cmbs,consultant,correspondent,foreign,master_servicer,...,investor_type_Life Insurance Company,investor_type_Pension Fund,investor_type_Private Investor/Developer,investor_type_Property Lender,investor_type_Property Owner,investor_type_REIT,investor_type_Sovereign Wealth Fund,investor_type_Special Servicer,investor_type_Tenants in Common,investor_type_Unknown
0,0012A000023XlCCQA0,1,2367,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,0012A000023XmOxQAK,9,7974,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,0012A000023YLqwQAG,891,50161,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0012A000023XlNjQAK,34,14631,1,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,0012A000023YBFRQA4,283,194195,1,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [363]:
test = pd.read_csv(r'C:\Users\Vincy Sagar\Desktop\Affine\AffineData\Dataset\Test dataset\Test_Data.csv')

In [364]:
test = test.merge(acc,on=['id_accs'],how='left')

In [365]:
test.shape

(29, 37)

In [291]:
#deals_data.head()

In [292]:
deals_data=deals.merge(ap_acc_prop[['id_props','lender_book']], on=['id_props'], how='left')
deals_data=deals.merge(ap_acc_prop[['id_props','loan_sales_book','investor_type_Advisor/Equity Fund','investor_type_Private Investor/Developer','investor_type_REIT']], on=['id_props'], how='left')



In [293]:
deals_data.head()

,id,createdbyid,id_deals,id_props,loan_sales_book,investor_type_Advisor/Equity Fund,investor_type_Private Investor/Developer,investor_type_REIT
0,a0A2A00000TxIrIUAV,005F0000007WnNDIA0,0062A00000o979GQAQ,a0I2A00000XQH8YUAX,0.0,0.0,0.0,1.0
1,a0A2A00000TxIrIUAV,005F0000007WnNDIA0,0062A00000o979GQAQ,a0I2A00000XQH8YUAX,1.0,0.0,1.0,0.0
2,a0A2A00000TxIrIUAV,005F0000007WnNDIA0,0062A00000o979GQAQ,a0I2A00000XQH8YUAX,1.0,0.0,0.0,0.0
3,a0A2A00000TxIrSUAV,005F0000007WnNDIA0,0062A00000o94JrQAI,a0I2A00000XQJ4dUAH,NaN,NaN,NaN,NaN
4,a0A2A00000TxIrXUAV,005F0000007WnNDIA0,0062A00000o94GJQAY,a0I2A00000XQJL0UAP,1.0,0.0,0.0,0.0


In [294]:
deals_data.describe()

,loan_sales_book,investor_type_Advisor/Equity Fund,investor_type_Private Investor/Developer,investor_type_REIT
count,31799.000000,31799.000000,31799.000000,31799.000000
mean,0.883770,0.146105,0.304412,0.080443
std,0.320506,0.353217,0.460165,0.271982
min,0.000000,0.000000,0.000000,0.000000
25%,1.000000,0.000000,0.000000,0.000000
50%,1.000000,0.000000,0.000000,0.000000
75%,1.000000,0.000000,1.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000


In [295]:
deals_data.drop('createdbyid', axis=1, inplace=True)

In [309]:
deals_data['loan_sales_book'].fillna(1.0, inplace=True)
deals_data['investor_type_Advisor/Equity Fund'].fillna(0.0, inplace=True)
deals_data['investor_type_Private Investor/Developer'].fillna(0.0, inplace=True)
deals_data['investor_type_REIT'].fillna(0.0, inplace=True)


In [310]:
deals_data.describe()

,loan_sales_book,investor_type_Advisor/Equity Fund,investor_type_Private Investor/Developer,investor_type_REIT
count,36875.000000,36875.000000,36875.000000,36875.000000
mean,0.899769,0.125993,0.262508,0.069369
std,0.300311,0.331846,0.440003,0.254085
min,0.000000,0.000000,0.000000,0.000000
25%,1.000000,0.000000,0.000000,0.000000
50%,1.000000,0.000000,0.000000,0.000000
75%,1.000000,0.000000,1.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000


In [311]:
deals_data['loan_sales_book'].dtype

dtype('float64')

In [289]:
deals.describe()

,id,createdbyid,id_deals,id_props
count,25820,25820,25820,25820
unique,25820,1,20350,19701
top,a0A2A00000TzqNLUAZ,005F0000007WnNDIA0,0062A00000q9jO9QAI,a0I2A00000XQHbQUAX
freq,1,25820,58,9


In [260]:
prop['sale_status'].fillna('N', inplace=True)

In [261]:
prop['sale_status'].unique()

array(['N', 'Y'], dtype=object)

In [262]:
prop['region__c'].fillna('Southwest', inplace=True)

In [298]:
prop['region__c'].unique()

array(['Southwest', 'Southeast', 'Northeast', 'West', 'Midwest',
       'Mid-Atlantic'], dtype=object)

In [299]:
deals_data=deals_data.merge(prop[['id_props','portfolio','sale_status','region__c']], on=['id_props'], how='left')

In [300]:
deals_data['portfolio'].unique()

array([False, True, nan], dtype=object)

In [301]:
deals_data['portfolio'].fillna(False, inplace=True)

In [302]:
deals_data['sale_status'].unique()

array(['N', 'Y', nan], dtype=object)

In [303]:
deals_data['sale_status'].fillna('N', inplace=True)

In [304]:
deals_data['region__c'].describe()

count         34153
unique            6
top       Southwest
freq          10404
Name: region__c, dtype: object

In [305]:
deals_data['region__c'].fillna('Southwest', inplace=True)

In [312]:
deals_data.head()

,id,id_deals,id_props,loan_sales_book,investor_type_Advisor/Equity Fund,investor_type_Private Investor/Developer,investor_type_REIT,portfolio,sale_status,region__c
0,a0A2A00000TxIrIUAV,0062A00000o979GQAQ,a0I2A00000XQH8YUAX,0.0,0.0,0.0,1.0,False,N,Southwest
1,a0A2A00000TxIrIUAV,0062A00000o979GQAQ,a0I2A00000XQH8YUAX,1.0,0.0,1.0,0.0,False,N,Southwest
2,a0A2A00000TxIrIUAV,0062A00000o979GQAQ,a0I2A00000XQH8YUAX,1.0,0.0,0.0,0.0,False,N,Southwest
3,a0A2A00000TxIrSUAV,0062A00000o94JrQAI,a0I2A00000XQJ4dUAH,1.0,0.0,0.0,0.0,False,N,West
4,a0A2A00000TxIrXUAV,0062A00000o94GJQAY,a0I2A00000XQJL0UAP,1.0,0.0,0.0,0.0,True,N,West


In [366]:
test_2=pd.DataFrame({'loan_sales_book':test['loan_sales_book'],'investor_type_Advisor/Equity Fund':test['investor_type_Advisor/Equity Fund'],'investor_type_Private Investor/Developer':test['investor_type_Private Investor/Developer'], 'investor_type_REIT':test['investor_type_REIT']})

In [367]:
test_2.head()

,investor_type_Advisor/Equity Fund,investor_type_Private Investor/Developer,investor_type_REIT,loan_sales_book
0,1,0,0,1
1,0,0,0,1
2,0,0,0,1
3,1,0,0,1
4,0,0,0,1


In [368]:
test_2.shape

(29, 4)

In [327]:
test['investor_type_Advisor/Equity Fund'].dtype

dtype('int64')

In [317]:
deals_data2=deals_data.copy()

In [331]:
deals_data2.head()

,loan_sales_book,investor_type_Advisor/Equity Fund,investor_type_Private Investor/Developer,investor_type_REIT
0,0,0,0,1
1,1,0,1,0
2,1,0,0,0
3,1,0,0,0
4,1,0,0,0


In [319]:
Id=deals_data['id']
id_props=deals_data['id_deals']
id_deals=deals_data['id_props']
port=deals_data['portfolio']
sale=deals_data['sale_status']
reg=deals_data['region__c']

In [322]:
deals_data2.drop(['id','id_deals','id_props','portfolio','sale_status','region__c'],axis=1, inplace=True)

In [329]:
deals_data2['loan_sales_book']=deals_data2['loan_sales_book'].astype('int64')

In [330]:
deals_data2['investor_type_Advisor/Equity Fund']=deals_data2['investor_type_Advisor/Equity Fund'].astype('int64')
deals_data2['investor_type_Private Investor/Developer']=deals_data2['investor_type_Private Investor/Developer'].astype('int64')
deals_data2['investor_type_REIT']=deals_data2['investor_type_REIT'].astype('int64')

In [333]:
from xgboost.sklearn import XGBClassifier
modelxgb_port = XGBClassifier(
 learning_rate =0.1,
 n_estimators=1000,
 max_depth=5,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
 seed=27)
modelxgb_port.fit(deals_data2, port)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=1000,
       n_jobs=1, nthread=4, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=27, silent=True,
       subsample=0.8)

In [336]:
deals_data2.columns

Index(['loan_sales_book', 'investor_type_Advisor/Equity Fund',
       'investor_type_Private Investor/Developer', 'investor_type_REIT'],
      dtype='object')

In [369]:
columnsTitles = ['loan_sales_book', 'investor_type_Advisor/Equity Fund',
       'investor_type_Private Investor/Developer', 'investor_type_REIT']

test_2=test_2.reindex(columns=columnsTitles)

In [371]:
test_2.columns

Index(['loan_sales_book', 'investor_type_Advisor/Equity Fund',
       'investor_type_Private Investor/Developer', 'investor_type_REIT'],
      dtype='object')

In [372]:
test_2.head()

,loan_sales_book,investor_type_Advisor/Equity Fund,investor_type_Private Investor/Developer,investor_type_REIT
0,1,1,0,0
1,1,0,0,0
2,1,0,0,0
3,1,1,0,0
4,1,0,0,0


In [373]:
pred_port=modelxgb_port.predict(test_2)

C:\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [344]:
#test_2['portfolio']=pred_port

In [353]:
test_2.head()

,loan_sales_book,investor_type_Advisor/Equity Fund,investor_type_Private Investor/Developer,investor_type_REIT
0,1,1,0,0
1,1,1,0,0
2,1,1,0,0
3,1,1,0,0
4,1,1,0,0


In [347]:
#test_port=test_2['portfolio']

In [348]:
#test_2.drop(['portfolio'],axis=1, inplace=True)

In [ ]:
###############################################TEST SALE STATUS

In [352]:
from xgboost.sklearn import XGBClassifier
modelxgb_ss = XGBClassifier(
 learning_rate =0.1,
 n_estimators=1000,
 max_depth=5,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
 seed=27)
modelxgb_ss.fit(deals_data2, sale)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=1000,
       n_jobs=1, nthread=4, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=27, silent=True,
       subsample=0.8)

In [374]:
pred_sale=modelxgb_ss.predict(test_2)

C:\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [350]:
deals_data2.head()

,loan_sales_book,investor_type_Advisor/Equity Fund,investor_type_Private Investor/Developer,investor_type_REIT
0,0,0,0,1
1,1,0,1,0
2,1,0,0,0
3,1,0,0,0
4,1,0,0,0


In [355]:
from xgboost.sklearn import XGBClassifier
modelxgb_reg = XGBClassifier(
 learning_rate =0.1,
 n_estimators=1000,
 max_depth=5,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'multi:softprop',
 num=6,
 nthread=4,
 scale_pos_weight=1,
 seed=27)
modelxgb_reg.fit(deals_data2, reg)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=1000,
       n_jobs=1, nthread=4, num=6, objective='multi:softprob',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=27, silent=True, subsample=0.8)

In [375]:
pred_reg=modelxgb_reg.predict(test_2)

C:\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [376]:
test_2['portfolio']=pred_port
test_2['sale_status']=pred_sale
test_2['region__c']=pred_reg

In [381]:
test_2.head()

,loan_sales_book,investor_type_Advisor/Equity Fund,investor_type_Private Investor/Developer,investor_type_REIT,portfolio,sale_status,region__c,id_accs
0,1,1,0,0,False,N,Southwest,0012A00002AXHJUQA5
1,1,0,0,0,False,N,Southwest,0012A000023XlWaQAK
2,1,0,0,0,False,N,Southwest,0012A000023Y4bYQAS
3,1,1,0,0,False,N,Southwest,0012A000023XlbuQAC
4,1,0,0,0,False,N,Southwest,0012A000023Xlh9QAC


In [380]:
test_2.shape

(29, 8)

In [386]:
deals_data.head()

,id_props,loan_sales_book,investor_type_Advisor/Equity Fund,investor_type_Private Investor/Developer,investor_type_REIT,portfolio,sale_status,region__c
0,a0I2A00000XQH8YUAX,0.0,0.0,0.0,1.0,False,N,Southwest
1,a0I2A00000XQH8YUAX,1.0,0.0,1.0,0.0,False,N,Southwest
2,a0I2A00000XQH8YUAX,1.0,0.0,0.0,0.0,False,N,Southwest
3,a0I2A00000XQJ4dUAH,1.0,0.0,0.0,0.0,False,N,West
4,a0I2A00000XQJL0UAP,1.0,0.0,0.0,0.0,True,N,West


In [379]:
test_2['id_accs'] = test['id_accs']

In [384]:
deals_data_orig = deals_data.copy()

In [385]:
deals_data.drop(['id','id_deals'],axis=1,inplace=True)

In [388]:
test_3 = test_2.merge(deals_data,on=['loan_sales_book','investor_type_Advisor/Equity Fund','investor_type_Private Investor/Developer','investor_type_REIT','portfolio','sale_status','region__c'],how='left')

In [390]:
test_3.shape

(69966, 9)

In [391]:
testsub_25_3 = pd.DataFrame({'id_accs':test_3['id_accs'],'id_prop':test_3['id_props']})

In [392]:
testsub_25_3.head()

,id_accs,id_prop
0,0012A00002AXHJUQA5,a0I2A00000XQGYEUA5
1,0012A00002AXHJUQA5,a0I2A00000XQLbCUAX
2,0012A00002AXHJUQA5,a0I2A00000XQHGZUA5
3,0012A00002AXHJUQA5,a0I2A00000XQHbLUAX
4,0012A00002AXHJUQA5,a0I2A00000XQH97UAH


In [217]:
oppo['property_type'].unique()

array(['Healthcare', 'Biotech', 'Seniors Housing', 'Hotel/Lodging',
       'Industrial', 'Land', 'Mixed Use', 'Apartments',
       'Manufactured Home Communities', 'SFR - Attached',
       'SFR - Detached', 'Office', 'Data Center', 'Unassigned',
       'Parking Garage', 'Recreational/Speciality', 'Promissory Note',
       'Day Care', 'Retail', 'Self Storage'], dtype=object)

In [216]:
oppo['property_type'].describe()
oppo['property_type'].fillna('Apartments', inplace=True)

In [219]:
deals_prop=deals_prop.merge(oppo[['id_deals','property_type']], on=['id_deals'], how='left')

In [220]:
l3=pd.DataFrame(deals_prop[deals_prop['property_type'].isnull()])
l3.shape

(10481, 7)

In [313]:
test.head()

,id_accs,active_deals,activity_count,sale_status,region__c,portfolio,buyer_book,servicing_contract,cmbs,consultant,...,investor_type_Pension Fund,investor_type_Private Investor/Developer,investor_type_Property Lender,investor_type_Property Owner,investor_type_REIT,investor_type_Sovereign Wealth Fund,investor_type_Special Servicer,investor_type_Tenants in Common,investor_type_Unknown,id_props
0,0012A00002AXHJUQA5,113,203365,N,Southeast,False,1,0,0,0,...,0,0,0,0,0,0,0,0,0,a0I2A00000XQKwXUAX
1,0012A00002AXHJUQA5,113,203365,N,Southeast,False,1,0,0,0,...,0,0,0,0,0,0,0,0,0,a0I2A00000XQGxsUAH
2,0012A00002AXHJUQA5,113,203365,N,Southeast,False,1,0,0,0,...,0,0,0,0,0,0,0,0,0,a0I2A00000XQIT1UAP
3,0012A00002AXHJUQA5,113,203365,N,Southeast,False,1,0,0,0,...,0,0,0,0,0,0,0,0,0,a0I2A00000XQGLrUAP
4,0012A00002AXHJUQA5,113,203365,N,Southeast,False,1,0,0,0,...,0,0,0,0,0,0,0,0,0,a0I2A00000XQGntUAH


In [360]:
test.head()

,id_accs,active_deals,activity_count,sale_status,region__c,portfolio,buyer_book,servicing_contract,cmbs,consultant,...,investor_type_Pension Fund,investor_type_Private Investor/Developer,investor_type_Property Lender,investor_type_Property Owner,investor_type_REIT,investor_type_Sovereign Wealth Fund,investor_type_Special Servicer,investor_type_Tenants in Common,investor_type_Unknown,id_props
0,0012A00002AXHJUQA5,113,203365,N,Southeast,False,1,0,0,0,...,0,0,0,0,0,0,0,0,0,a0I2A00000XQKwXUAX
1,0012A00002AXHJUQA5,113,203365,N,Southeast,False,1,0,0,0,...,0,0,0,0,0,0,0,0,0,a0I2A00000XQGxsUAH
2,0012A00002AXHJUQA5,113,203365,N,Southeast,False,1,0,0,0,...,0,0,0,0,0,0,0,0,0,a0I2A00000XQIT1UAP
3,0012A00002AXHJUQA5,113,203365,N,Southeast,False,1,0,0,0,...,0,0,0,0,0,0,0,0,0,a0I2A00000XQGLrUAP
4,0012A00002AXHJUQA5,113,203365,N,Southeast,False,1,0,0,0,...,0,0,0,0,0,0,0,0,0,a0I2A00000XQGntUAH


In [395]:
dataframe=acc.merge(acc_prop[['id_accs','id_props']],on=['id_accs'], how='left')

In [4]:
test_value_count = pd.read_csv(r'C:\Users\Vincy Sagar\Desktop\Affine\affine_26\testvalue_freq.csv')

In [5]:
test_value_count.head(10)

,id_prop,freq
0,a0I2A00000XQGxiUAH,17
1,a0I2A00000XQJUfUAP,17
2,a0I2A00000YGe5OUAT,15
3,a0I2A00000XQINQUA5,15
4,a0I2A00000XQHX7UAP,15
5,a0I2A00000XQKAwUAP,15
6,a0I2A00000XQJXtUAP,15
7,a0I2A00000XQGTeUAP,15
8,a0I2A00000XQM7dUAH,15
9,a0I2A00000XQJh5UAH,15


In [403]:
#########################################    Filtering id_prop according to frequency

In [406]:
#################################Filtering id3
id3=pd.DataFrame(test_value_concat[test_value_concat['id_accs']=='0012A000023Y4bYQAS'])
#id3.drop(id3.index[-100:], inplace=True)
id3.shape
id3 = id3.merge(test_concat,on=['id_prop'],how='left')
id3temp = pd.DataFrame(id3[ (id3['freq']==17)])
id3.sort_values(by='freq', ascending=False,inplace=True)
id3.reset_index(drop=True,inplace=True)
drop_indices = np.random.choice(2995, 1500, replace=False)
id3 = id3.drop(drop_indices)

#################################Filtering id4#################################Filtering id4

id4=pd.DataFrame(test_value_concat[test_value_concat['id_accs']=='0012A000023XlbuQAC'])
id4.drop(id34index[-100:], inplace=True)
id4.shape
drop_indices = np.random.choice(id4.index, 500, replace=False)
id4 = id4.drop(drop_indices)
id4 = id4.merge(test_concat,on=['id_prop'],how='left')

#################################Filtering id5#################################Filtering id5

id5=pd.DataFrame(test_value_concat[test_value_concat['id_accs']=='0012A000023Xlh9QAC'])
id5.drop(id5.index[-100:], inplace=True)
id5 = id5.merge(test_concat,on=['id_prop'],how='left')
id5.shape
id5temp17 = pd.DataFrame(id5[(id5['freq']==17)])
id5temp17.shape

#################################Filtering id6#################################Filtering id6

id6=pd.DataFrame(test_value_concat[test_value_concat['id_accs']=='0012A000023XlhdQAC'])
id6.drop(id6.index[-100:], inplace=True)
id6 = id6.merge(test_concat,on=['id_prop'],how='left')
id6.shape
id6temp17 = pd.DataFrame(id6[(id6['freq']==17)])
id6temp17.shape

########################################################################################id7

id7=pd.DataFrame(test_value_concat[test_value_concat['id_accs']=='0012A000023XlkNQAS'])
id7.drop(id7.index[-100:], inplace=True)
id7 = id7.merge(test_concat,on=['id_prop'],how='left')
id7.shape
drop_indices = np.random.choice(id7.index, 1200, replace=False)
id7 = id7.drop(drop_indices)

#######################################################id 8

id8=pd.DataFrame(test_value_concat[test_value_concat['id_accs']=='0012A000023Y4bYQAS'])
#id3.drop(id3.index[-100:], inplace=True)
id8.shape
id8 = id8.merge(test_concat,on=['id_prop'],how='left')
id8temp = pd.DataFrame(id8[ (id8['freq']==17)])
id8.sort_values(by='freq', ascending=False,inplace=True)
id8.reset_index(drop=True,inplace=True)
drop_indices = np.random.choice(2995, 1500, replace=False)
id8 = id8.drop(drop_indices)

#################################Filtering id9#################################Filtering id9

id9=pd.DataFrame(test_value_concat[test_value_concat['id_accs']=='0012A000023XlbuQAC'])
id9.drop(id9index[-100:], inplace=True)
id9.shape
drop_indices = np.random.choice(id9.index, 500, replace=False)
id9 = id9.drop(drop_indices)
id9 = id9.merge(test_concat,on=['id_prop'],how='left')

#################################Filtering id10#################################Filtering id10

id10=pd.DataFrame(test_value_concat[test_value_concat['id_accs']=='0012A000023Xlh9QAC'])
id10.drop(id10.index[-100:], inplace=True)
id10 = id10.merge(test_concat,on=['id_prop'],how='left')
id10.shape
id10temp17 = pd.DataFrame(id10[(id10['freq']==17)])
id10temp17.shape
#################################Filtering id11#################################Filtering id11

id11=pd.DataFrame(test_value_concat[test_value_concat['id_accs']=='0012A000023Xm35QAC'])
#id3.drop(id3.index[-100:], inplace=True)
id11 = id11.merge(test_concat,on=['id_prop'],how='left')
id11.shape
drop_indices = np.random.choice(id11.index, 500, replace=False)
id11 = id11.drop(drop_indices)

In [7]:
#####################################################################id12
id12=pd.DataFrame(test_value_concat[test_value_concat['id_accs']=='0012A000023YB67QAG'])
#id3.drop(id3.index[-100:], inplace=True)
id12 = id12.merge(test_concat,on=['id_prop'],how='left')
id12.shape
drop_indices = np.random.choice(id12.index, 713, replace=False)
id12 = id12.drop(drop_indices)

###################################################id13

id13=pd.DataFrame(test_value_concat[test_value_concat['id_accs']=='0012A000023XmRwQAK'])
#id3.drop(id3.index[-100:], inplace=True)
id13 = id13.merge(test_concat,on=['id_prop'],how='left')
id13.shape
id13temp17 = pd.DataFrame(id13[(id13['freq']==17)])
id13temp17.shape

################################################################id14
id14=pd.DataFrame(test_value_concat[test_value_concat['id_accs']=='0012A000023XmmFQAS'])
#id3.drop(id3.index[-100:], inplace=True)
id14 = id14.merge(test_concat,on=['id_prop'],how='left')
id14.shape
###################################################id15
id15=pd.DataFrame(test_value_concat[test_value_concat['id_accs']=='0012A000023Y1rtQAC'])
#id3.drop(id3.index[-100:], inplace=True)
id15 = id15.merge(test_concat,on=['id_prop'],how='left')
id15.shape
drop_indices = np.random.choice(id15.index, 1500, replace=False)
id15 = id15.drop(drop_indices)

#############################################################################id 16



id16=pd.DataFrame(test_value_concat[test_value_concat['id_accs']=='0012A000023YBFRQA4'])
#id3.drop(id3.index[-100:], inplace=True)
id16 = id16.merge(test_concat,on=['id_prop'],how='left')
id16.shape
id16temp10 = pd.DataFrame(id16[(id16['freq']==17)])
id16temp10.shape

############################################################################################################ id 17

id17=pd.DataFrame(test_value_concat[test_value_concat['id_accs']=='0012A000023Xo0OQAS'])
id3.drop(id3.index[-100:], inplace=True)
id17 = id17.merge(test_concat,on=['id_prop'],how='left')
id17.shape
drop_indices = np.random.choice(id17temp17.index, 1800, replace=False)
id17temp17 = id17temp17.drop(drop_indices)

############################################################################################################ id 18

id18=pd.DataFrame(test_value_concat[test_value_concat['id_accs']=='0012A000023XoRjQAK'])
#id3.drop(id3.index[-100:], inplace=True)
#id18 = id18.merge(test_concat,on=['id_prop'],how='left')
id18.shape
id18temp17 = pd.DataFrame(id18[(id18['freq']==17)])
drop_indices = np.random.choice(id18temp17.index, 1800, replace=False)
id18temp17 = id18temp17.drop(drop_indices)

#################################################################################id19

id19=pd.DataFrame(test_value_concat[test_value_concat['id_accs']=='0012A000023XwMRQA0'])
#id3.drop(id3.index[-100:], inplace=True)
id19 = id19.merge(test_concat,on=['id_prop'],how='left')
id19.shape
drop_indices = np.random.choice(id19.index, 1513, replace=False)
id19 = id19.drop(drop_indices)

##############################################################################id20

id20=pd.DataFrame(test_value_concat[test_value_concat['id_accs']=='0012A000029ZzX1QAK'])
#id3.drop(id3.index[-100:], inplace=True)
id20 = id20.merge(test_concat,on=['id_prop'],how='left')
id20.shape
drop_indices = np.random.choice(id20.index, 1513, replace=False)
id20 = id20.drop(drop_indices)

######################################################################id 21

id21=pd.DataFrame(test_value_concat[test_value_concat['id_accs']=='0012A000023YLFWQA4'])
#id3.drop(id3.index[-100:], inplace=True)
id21 = id21.merge(test_concat,on=['id_prop'],how='left')
id21.shape
drop_indices = np.random.choice(id21.index, 713, replace=False)
id21 = id21.drop(drop_indices)

############################################################id 22

id22=pd.DataFrame(test_value_concat[test_value_concat['id_accs']=='0012A000023Xp4LQAS'])
#id3.drop(id3.index[-100:], inplace=True)
id22 = id22.merge(test_concat,on=['id_prop'],how='left')
id22.shape
drop_indices = np.random.choice(id22temp2.index, 3000, replace=False)
id22temp2 = id22temp2.drop(drop_indices)

#######################################################################id 23

id23=pd.DataFrame(test_value_concat[test_value_concat['id_accs']=='0012A000023XrTNQA0'])
#id3.drop(id3.index[-100:], inplace=True)
id23 = id23.merge(test_concat,on=['id_prop'],how='left')
id23.shape

################################################################################id 24

id24=pd.DataFrame(test_value_concat[test_value_concat['id_accs']=='0012A00002JF3YmQAL'])
#id3.drop(id3.index[-100:], inplace=True)
id24 = id24.merge(test_concat,on=['id_prop'],how='left')
id24.shape
id24temp10 = pd.DataFrame(id24[(id24['freq']==10)])
id24temp10.shape
drop_indices = np.random.choice(id24temp10.index, 2000, replace=False)
id24temp10 = id24temp10.drop(drop_indices)

########################################################################################id 25

id25=pd.DataFrame(test_value_concat[test_value_concat['id_accs']=='0012A000023YKsuQAG'])
#id3.drop(id3.index[-100:], inplace=True)
id25 = id25.merge(test_concat,on=['id_prop'],how='left')
id25.shape
id25temp5 = pd.DataFrame(id25[(id25['freq']==5)])
id25temp5.shape

#############################################################################################id 26

id26=pd.DataFrame(test_value_concat[test_value_concat['id_accs']=='0012A000023Y9gNQAS'])
#id3.drop(id3.index[-100:], inplace=True)
id26 = id26.merge(test_concat,on=['id_prop'],how='left')
id26.shape
drop_indices = np.random.choice(id26.index, 1313, replace=False)
id26 = id26.drop(drop_indices)

#############################################################################id 27

id27=pd.DataFrame(test_value_concat[test_value_concat['id_accs']=='0012A000026WwqxQAC'])
#id3.drop(id3.index[-100:], inplace=True)
id27 = id27.merge(test_concat,on=['id_prop'],how='left')
id27.shape
drop_indices = np.random.choice(id27.index, 2513, replace=False)
id27 = id27.drop(drop_indices)

############################################################################id 28

id28=pd.DataFrame(test_value_concat[test_value_concat['id_accs']=='0012A000023YAupQAG'])
#id3.drop(id3.index[-100:], inplace=True)
id28 = id28.merge(test_concat,on=['id_prop'],how='left')
id28.shape
drop_indices = np.random.choice(id28.index, 1713, replace=False)
id28 = id28.drop(drop_indices)

###############################################################################################id 29

id29=pd.DataFrame(test_value_concat[test_value_concat['id_accs']=='0012A000023YDnWQAW'])
#id3.drop(id3.index[-100:], inplace=True)
id29 = id29.merge(test_concat,on=['id_prop'],how='left')
id29.shape
drop_indices = np.random.choice(id29temp5.index, 1200, replace=False)
id29temp5 = id29temp5.drop(drop_indices)

In [16]:
########concating all
test_new_1=pd.concat([id1,id2,id3,id4,id5,id6,id7,id8,id9,id10,id11,id12,id13,id14,id15,id16,id17,id18,id19,id20,id21,id22,id23,id24,id25,id26,id27,id28,id29], axis=0)

In [17]:
test_new_1.to_csv('TestFinalSub.csv')